# Progress of the Philippines' Sustainable Development Goals

### Import

In [603]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore

## Data Collection
The following **csv** files used in this project are acquired through a request sent to the Knowledge Management and Communications Division of the Philippine Statistics Authority.

### Combining the Datasets 
In this stage, the separate datasets underwent pre-processing and cleaning before they are combined together. 

First, the irrelevant rows were dropped first. These were the rows that have all NaN values and the additional rows (i.e., note rows, “Data available” rows) found in the CSV files. 

Second, since the first row of the CSV files was the name of the indicator and unnamed rows, the resulting DataFrame had “Unnamed” as its column header. Due to this, we had to set the column headers to the second row of the DataFrame, and then drop this afterward.

Third, since the `Geolocation` column would be used later to merge the datasets, the values in this column were standardized into the format `Region n: region_name`, where *n* is the corresponding region number and *region_name* is the name of the region. If it does not have a region number, then it was formatted as `region_abbreviation: region_name`, where *region_abbreviation* is its official abbreviation. 

Fourth, there are datasets that had divisions for a region and year, but still include a cumulative value for that division (e.g., datasets that are also divided per `Sex`, while having a value of “Both Sexes”. For this situation, we have decided to only get the cumulative row (e.g., Both Sexes), drop the other rows that represent the division (e.g., Female and Male), and drop the column that is related to this division (Sex). 

Fifth, we convert the DataFrame into its long representation. Once we have the dataset into its long representation, then we can merge it to the combined dataset while using the Year and Geolocation columns as its primary key. This would be done for all of the twenty-five datasets.

This process would result in one DataFrame that is in its long representation, with three kinds of columns: (1) Geolocation, (2) Year, and (3) the value for each of the indicators. 

#### 1.2.1. Proportion of population living below the national poverty line 
To start with, let us load the data from the csv file using pandas' [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function.

The [`os.getenv`](https://docs.python.org/3/library/os.html) function was used to get the environment variable `DSDATA_PROJ`, which points to the data folder of this project.

In [604]:
data = pd.read_csv('data' + '/1.2.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/1.2.1.csv')
data

,1.2.1 Proportion of population living below the national poverty line by sex age 1/,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015.0,2016,2017,2018.0,2019,2020,2021,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,..,..,..,..,..,..,..,..,...,..,..,23.5,..,..,16.7,..,..,...,..
3,..National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,4.1,..,..,2.2,..,..,...,..
4,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,22.7,..,..,12.0,..,..,...,..
5,..Region I,..,..,..,..,..,..,..,..,..,...,..,..,18.8,..,..,9.9,..,..,...,..
6,..Region II,..,..,..,..,..,..,..,..,..,...,..,..,17.8,..,..,16.3,..,..,...,..
7,..Region III,..,..,..,..,..,..,..,..,..,...,..,..,10.5,..,..,7.0,..,..,...,..
8,..Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,12.5,..,..,7.1,..,..,...,..
9,..MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,25.2,..,..,15.1,..,..,...,..


Looking at the DataFrame, we could see that the columns are unnamed and that the column names are located at the 0th row. Using [`iloc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html), we could get the 0th row and then assign it as the column values. 

Then, using the [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) function, we can drop the 0th row as we have no need for it anymore. Additionally, since the row at index 1 is a row full of NaN, we can also drop it using the same function. 

To be able to fix the indexing of the rows, the [`reset_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) function was used to reset the index from 0.

In [605]:
# setting our column names
data.columns = data.iloc [0] 

# dropping the 'geolocation' row as that is actually used as a header
data = data.drop (data.index [1])

# dropping the column names 
data = data.drop (data.index [0])

data.reset_index (drop=True, inplace=True)

Irrelevant rows that are just footers for the file are also removed.

In [606]:
# dropping irrelevant rows 
data = data.drop (data.index [18:]) 

The `Year` column must also be renamed into `Geolocation` as this row refers to the different regions in the Philippines, and not the years. This can be done through the use of the of the [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) function.

In [607]:
# renames the column 'Year' as its actually the location column
data.rename(columns = {'Year':'Geolocation'}, inplace=True)

To easily determine which region the `Geolocation` values refer to, we can also change these values to include the names that they are commonly referred to, instead of just their region numbers. 

For consistency throughout the different datasets, the `region_names` variable was declared. The reason why a map was not used was that different datasets have different representations of the region (i.e., differences in naming a region), however, they are always arranged in the same way. This would be shown below in the pre-processing of each of the datasets.

In [608]:
# NOTE: Before applying, make sure that the arrangement of the regions are the same as the arrangement in your table
region_names = ['PHILIPPINES', 'NCR: National Capital Region', 
                 'CAR: Cordillera Administrative Region', 
                 'Region 1: Ilocos Region', 
                 'Region 2: Cagayan Valley', 
                 'Region 3: Central Luzon', 
                 'Region 4A: CALABARZON', 
                'MIMAROPA: Southwestern Tagalog Region', 
                'Region 5: Bicol Region', 
                'Region 6: Western Visayas', 
                'Region 7: Central Visayas', 
                'Region 8: Eastern Visayas', 
                'Region 9: Zamboanga Peninsula', 
                'Region 10: Northern Mindanao', 
                'Region 11: Davao Region', 
                'Region 12: SOCCSKSARGEN', 
                'CARAGA: Cordillera Administrative Region', 
                'BARMM: Bangsamoro Autonomous Region in Muslim Mindanao']

In [609]:
# renames the data in the Geolocation for consistency
data['Geolocation'] = region_names
data.set_index('Geolocation')
data = data.reset_index(drop=True)
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015.0,2016,2017,2018.0,2019,2020,2021,2022
0,PHILIPPINES,..,..,..,..,..,..,..,..,..,...,..,..,23.5,..,..,16.7,..,..,...,..
1,NCR: National Capital Region,..,..,..,..,..,..,..,..,..,...,..,..,4.1,..,..,2.2,..,..,...,..
2,CAR: Cordillera Administrative Region,..,..,..,..,..,..,..,..,..,...,..,..,22.7,..,..,12.0,..,..,...,..
3,Region 1: Ilocos Region,..,..,..,..,..,..,..,..,..,...,..,..,18.8,..,..,9.9,..,..,...,..
4,Region 2: Cagayan Valley,..,..,..,..,..,..,..,..,..,...,..,..,17.8,..,..,16.3,..,..,...,..
5,Region 3: Central Luzon,..,..,..,..,..,..,..,..,..,...,..,..,10.5,..,..,7.0,..,..,...,..
6,Region 4A: CALABARZON,..,..,..,..,..,..,..,..,..,...,..,..,12.5,..,..,7.1,..,..,...,..
7,MIMAROPA: Southwestern Tagalog Region,..,..,..,..,..,..,..,..,..,...,..,..,25.2,..,..,15.1,..,..,...,..
8,Region 5: Bicol Region,..,..,..,..,..,..,..,..,..,...,..,..,39.8,..,..,27.0,..,..,...,..
9,Region 6: Western Visayas,..,..,..,..,..,..,..,..,..,...,..,..,24.6,..,..,16.3,..,..,...,..


Next, we can convert the strings of '..' and '...', which were used to represent that there were no values for these cells, to **NaN**, through the use of the [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) function.

However, the columns that have all **NaN** values were not dropped because if this dataset would be combined with other datasets, all years would still be present as there are datasets with complete data for all the years. Additionally, dropping the years for some of the dataset would result in the combined dataset having a weird sorting (i.e., a sorting of the region that does not follow the usual sorting of the datasets in the Philippines), even if it was sorted based on the `Year` and `Geolocation` column.

In [610]:
for c in data.columns.difference(['Geolocation']):
    # cells without values are represented as either '..' or '...', so we should convert them to NaN so we could dropna()
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# drops columns if all of the values are NaN
# data = data.dropna(axis=1)

In [611]:
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015.0,2016,2017,2018.0,2019,2020,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23.5,NaN,NaN,16.7,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.1,NaN,NaN,2.2,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,22.7,NaN,NaN,12.0,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,18.8,NaN,NaN,9.9,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.8,NaN,NaN,16.3,NaN,NaN,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.5,NaN,NaN,7.0,NaN,NaN,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.5,NaN,NaN,7.1,NaN,NaN,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,25.2,NaN,NaN,15.1,NaN,NaN,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,39.8,NaN,NaN,27.0,NaN,NaN,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,24.6,NaN,NaN,16.3,NaN,NaN,NaN,NaN


As the final step, the wide representation of this dataset is converted to a long representation through the use of the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function. 

Then, the column that holds the value for a specific year and region is coverted, using [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html), to the ID of this Sustainable Development Goal (SDG), so that it can be distinguished when it is combined with other datasets.

In [612]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'1.2.1. Proportion of population living below the national poverty line', 0 : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


As this is the first dataset, we can just assign it to the `combined_data` DataFrame, which would hold the combined datasets.

In [613]:
combined_data = data

#### 1.4.1p5. Net Enrolment Rate in elementary

Using the same [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function, we load the next dataset. 

In [614]:
data = pd.read_csv('data' + '/1.4.1p5.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/1.4.1p5.csv')
data

,1.4.1p5 Net Enrolment Rate in elementary (Indicator is also found in SDG 4.3.s1) 1/,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,NaN,Year,2000,2001,2002.00,2003.00,2004.00,2005.00,2006.00,2007.00,...,2013.00,2014.00,2015.00,2016.00,2017.00,2018.00,2019.00,2020.0000,2021,2022
1,Geolocation,Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,Both Sexes,96.77,90.1,90.29,88.74,87.11,84.44,83.22,84.93,...,97.20,97.19,96.90,96.15,94.19,94.05,93.96,89.1064,...,...
3,NaN,Boys,96.27,89.33,89.51,87.84,86.17,83.56,82.39,84.07,...,96.74,96.87,96.66,96.17,94.12,94.25,93.79,88.9318,...,...
4,NaN,Girls,97.28,90.91,91.10,89.68,88.08,85.35,84.08,85.83,...,97.68,97.53,97.15,96.12,94.27,93.85,94.15,89.2898,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,.. - Data not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,... - Data not yet available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,1/ - Updates were based on the submission of D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From the DataFrame above, we can see that the footer of the .csv files was included in the DataFrame. As the rows from the 56th index are irrelevant, we can [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) them. 

In [615]:
data = data.drop (data.index [56:]) 

Additionally, we can see that the columns are unnamed, and upon inspection, the original column names can be found at `Index 0`. Thus, we can set the columns to this row, and then  [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the `Index 0` row as it would only be redundant and might affect the computations.

The [`reset_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) function was used in order to make the index of the rows start from 0.

In [616]:
# setting the column names and removing the row that held the previous column names
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)
data

,NaN,Year,2000,2001,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021,2022
0,Geolocation,Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PHILIPPINES,Both Sexes,96.77,90.1,90.29,88.74,87.11,84.44,83.22,84.93,...,97.20,97.19,96.90,96.15,94.19,94.05,93.96,89.1064,...,...
2,NaN,Boys,96.27,89.33,89.51,87.84,86.17,83.56,82.39,84.07,...,96.74,96.87,96.66,96.17,94.12,94.25,93.79,88.9318,...,...
3,NaN,Girls,97.28,90.91,91.10,89.68,88.08,85.35,84.08,85.83,...,97.68,97.53,97.15,96.12,94.27,93.85,94.15,89.2898,...,...
4,..National Capital Region (NCR),Both Sexes,101,97.82,97.38,96.81,94.82,92.61,92.89,94.42,...,99.64,99.01,99.85,95.92,92.83,92.11,89.91,81.1478,...,...
5,NaN,Boys,100.13,96.57,96.52,95.81,93.75,91.65,92.00,93.21,...,98.77,98.13,98.80,95.30,92.20,91.85,89.43,80.6316,...,...
6,NaN,Girls,101.92,99.13,98.28,97.87,95.95,93.63,93.83,95.69,...,100.57,99.95,100.95,96.58,93.50,92.38,90.42,81.6903,...,...
7,..Cordillera Administrative Region (CAR),Both Sexes,94.42,92.89,91.52,89.19,86.40,82.58,80.86,81.50,...,99.66,100.16,99.19,97.24,94.37,92.24,91.40,87.5276,...,...
8,NaN,Boys,94.26,91.96,90.53,88.36,85.52,81.75,80.19,81.01,...,99.85,100.27,99.42,97.94,95.13,93.45,92.25,88.5518,...,...
9,NaN,Girls,94.58,93.88,92.57,90.07,87.31,83.46,81.57,82.01,...,99.47,100.05,98.95,96.51,93.59,90.99,90.51,86.4657,...,...


However, these is still a row of NaN found at `Index 0`, and we can see that the column names for the first two columns are not correct for the values underneath it, as the ones under the first column are actually Geolocations and those under the second columns are the values for Sex. Thus, we can [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) it, and then [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)  the row at `Index 0`.

In [617]:
data = data.rename(columns = {np.nan:'Geolocation', 'Year': 'Sex'})
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

As we would only need the data that is grouped by region and not by sex, we would only be getting the rows that has **Both Sexes** as the value in the Sex column. After this, we can [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the Sex column as it would not be used onwards.

In [618]:
# Only getting the total data, then dropping Sex column as it's not needed anymore
data = data[data['Sex'] == 'Both Sexes']
data = data.drop("Sex", axis = 1)
data = data.reset_index (drop=True)
data

,Geolocation,2000,2001,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021,2022
0,PHILIPPINES,96.77,90.1,90.29,88.74,87.11,84.44,83.22,84.93,85.11,...,97.20,97.19,96.90,96.15,94.19,94.05,93.96,89.1064,...,...
1,..National Capital Region (NCR),101,97.82,97.38,96.81,94.82,92.61,92.89,94.42,93.69,...,99.64,99.01,99.85,95.92,92.83,92.11,89.91,81.1478,...,...
2,..Cordillera Administrative Region (CAR),94.42,92.89,91.52,89.19,86.40,82.58,80.86,81.50,81.93,...,99.66,100.16,99.19,97.24,94.37,92.24,91.40,87.5276,...,...
3,..Region I,97.73,91.33,89.64,88.52,86.98,84.87,82.74,83.14,82.85,...,97.39,97.84,96.78,94.84,92.50,90.48,89.99,86.2185,...,...
4,..Region II,95.65,89.45,86.71,85.65,82.90,79.92,77.70,77.53,76.23,...,100.08,101.15,102.42,100.26,98.45,96.86,97.17,93.6348,...,...
5,..Region III,98.32,86.35,93.58,93.61,92.03,90.77,89.14,91.37,90.93,...,99.03,99.56,98.84,98.53,97.91,98.77,100.03,95.4067,...,...
6,..Region IV-A 2/,98.5,93.44,95.97,95.33,95.10,92.87,92.36,94.02,94.10,...,96.09,97.09,96.36,97.20,96.31,97.36,98.23,91.9912,...,...
7,..MIMAROPA 2/,..,..,91.52,89.42,88.00,84.39,83.84,84.07,85.42,...,95.77,95.58,96.56,94.98,92.33,90.99,90.26,86.2074,...,...
8,..Region V,95.56,91.77,90.95,89.30,87.78,85.43,83.80,85.41,85.07,...,98.53,98.25,96.41,95.77,93.56,93.12,92.68,87.2573,...,...
9,..Region VI,96.16,89.6,85.95,83.25,80.49,77.14,74.96,75.44,74.93,...,97.71,98.47,98.89,99.09,97.16,97.38,97.25,93.9281,...,...


To be able to merge this to the combined DataFrame, the value of the Geolocation column has been set to the same values.

In [619]:
data['Geolocation'] = region_names

Since the dataset represents missing values as either '...' or '..', we can [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) the columns with these values with `np.nan`.

In [620]:
for c in data.columns.difference(['Geolocation']):
    # cells without values are represented as either '..' or '...', so we should convert them to NaN so we could dropna()
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')

Then, we can transform the wide representation of the DataFrame to its long representation version using the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function. 

In [621]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'1.4.1p5 Net Enrolment Rate in elementary', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

In [622]:
data

,Geolocation,Year,1.4.1p5 Net Enrolment Rate in elementary
0,PHILIPPINES,2000,96.77
1,NCR: National Capital Region,2000,101
2,CAR: Cordillera Administrative Region,2000,94.42
3,Region 1: Ilocos Region,2000,97.73
4,Region 2: Cagayan Valley,2000,95.65
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


Then we can [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) this long representation DataFrame into the combined DataFrame. It would be merged with respect to the values in the **Geolocation** and **Year** column. An outer join is used as we want to retain all the values of both of the DataFrames, even if there would be **NaN** values for some of cells.

In [623]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [624]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary
0,PHILIPPINES,2000,NaN,96.77
1,NCR: National Capital Region,2000,NaN,101
2,CAR: Cordillera Administrative Region,2000,NaN,94.42
3,Region 1: Ilocos Region,2000,NaN,97.73
4,Region 2: Cagayan Valley,2000,NaN,95.65
...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN


#### 1.4.1p6. Net Enrolment Rate in secondary education (Indicator is also found in SDG 4.3.s2)

Next, we can load the third dataset.

In [625]:
data = pd.read_csv('data' + '/1.4.1p6.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/1.4.1p6.csv')
data

,1.4.1p6 Net Enrolment Rate in secondary education (Indicator is also found in SDG 4.3.s2),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,NaN,Year,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016.00,2017.00,2018.00,2019.00,2020.0000,2021,2022
1,Level of Education,Geolocation,Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Junior High School,PHILIPPINES,Both Sexes,66.06,57.55,59,60.15,59.97,58.54,58.59,...,67.89,67.19,73.57,74.19,75.99,81.41,82.89,81.4869,...,...
3,NaN,NaN,Boys,62.72,52.96,54.39,55.34,55.04,53.65,53.85,...,62.42,61.68,68.09,68.79,70.88,77.24,78.80,77.6557,...,...
4,NaN,NaN,Girls,69.49,62.24,63.72,65.07,65.01,63.53,63.44,...,73.69,73.05,79.42,79.94,81.42,85.82,87.20,85.5003,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,.. - Data not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,... - Data not yet available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,1/ - Updates were based on submission of DepEd...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2/ - Estimation of this sub-indicator only sta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Just like in the processing of the previous datasets, we first [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the unnecessary rows at the bottom part of the DataFrame. 

In [626]:
data = data.drop (data.index [110:]) 

From the DataFrame above, we can see that the correct column headers are found at `Index 0`. However, upon inspection, we would see that there are two NaN values and the 'Year' value at the third column should actually be 'Sex' based on the values below it. Thus, before setting this row as the column header, we first correct the values of these first three columns using the [`at`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) function.

In [627]:
data.at[0, '1.4.1p6 Net Enrolment Rate in secondary education (Indicator is also found in SDG 4.3.s2)'] = 'Level of Education'
data.at[0, 'Unnamed: 1'] = 'Geolocation'
data.at[0, 'Unnamed: 2'] = 'Sex'

Now that first row can correctly act as the column header, we can set is as the column header, before dropping the row at `Index 0`. Then we must also [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the row of **NaN**s at `Index 1` as it is unnecessary, before using the [`reset_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) function.

In [628]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

Using the [`unique`](https://pandas.pydata.org/docs/reference/api/pandas.unique.html) function, we can see that there are two values for 'Level of Education' columns. To be able to combine this to the combined dataset, we must separate them as we cannot add another column that would hold the education level, thus, we can just add it as two different columns.

In [629]:
data ['Level of Education'].unique ()

array(['Junior High School', nan, 'Senior High School'], dtype=object)

In [630]:
senior_high_data = data [54:]
junior_high_data = data [:54]

Now, we must process these two separately, but the processes done to them would be the same.

First, as we only need the general data, without taking *Sex* into consideration. This can be done by only getting the rows that has **Both Sexes** as the value of the `Sex` column.

In [631]:
junior_high_data = junior_high_data [junior_high_data['Sex'] == 'Both Sexes']
junior_high_data = junior_high_data.reset_index (drop=True)

In [632]:
senior_high_data = senior_high_data [senior_high_data['Sex'] == 'Both Sexes']
senior_high_data = senior_high_data.reset_index (drop=True)

Next, as we have already separated the dataset into two based on the value of the `Level of Education` column, we have no need for this column anymore. This means that we can [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) this column.  

In [633]:
junior_high_data = junior_high_data.drop("Level of Education", axis = 1)
junior_high_data = junior_high_data.drop("Sex", axis = 1)
junior_high_data = junior_high_data.reset_index (drop=True)

In [634]:
senior_high_data = senior_high_data.drop("Level of Education", axis = 1)
senior_high_data = senior_high_data.drop("Sex", axis = 1)
senior_high_data = senior_high_data.reset_index (drop=True)

For consistency, we set the values of the `Geolocation` column to the format of the region names that we have decided before.

In [635]:
senior_high_data['Geolocation'] = region_names

In [636]:
junior_high_data['Geolocation'] = region_names

As the dataset represents missing values as '..' or '...', we must [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) these values with `np.nan`.

In [637]:
for c in junior_high_data.columns.difference(['Geolocation']):
    junior_high_data [c].replace(to_replace='..', value= np.nan, inplace= True)
    junior_high_data [c].replace(to_replace='...', value= np.nan, inplace= True)

In [638]:
for c in senior_high_data.columns.difference(['Geolocation']):
    senior_high_data [c].replace(to_replace='..', value= np.nan, inplace= True)
    senior_high_data [c].replace(to_replace='...', value= np.nan, inplace= True)

Looking at the senior high data, we can see that all of the values are `NaN` from 2000 to 2016, which is to be expected as Senior High School was only implemented from 2016.

In [639]:
senior_high_data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.0,2017.0,2018.0,2019.0,2020.0,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,37.38,46.12,51.24,47.76,49.4800,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,55.32,62.74,68.63,62.28,56.4435,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,40.16,49.55,53.64,50.53,52.8763,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,51.11,60.39,64.06,61.54,65.6379,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,43.41,51.49,56.21,56.46,61.4433,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,47.96,55.99,60.19,58.03,60.0165,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,45.61,53.90,58.33,54.79,54.7999,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,35.09,43.27,48.14,46.00,50.2024,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,28.35,39.63,45.80,42.31,43.5180,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,32.54,44.17,49.74,44.22,48.2144,NaN,NaN


Next, we can convert both of the datasets into its long representation using the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function.

In [640]:
junior_high_data = pd.melt(junior_high_data, id_vars='Geolocation', value_vars=junior_high_data.columns [1:]) 

junior_high_data.rename(columns = {'value':'1.4.1p6 Net Enrolment Rate in secondary education (Junior High School)', 0 : 'Year'}, inplace=True)
junior_high_data = junior_high_data.astype({'Year':'int'})

In [641]:
senior_high_data = pd.melt(senior_high_data, id_vars='Geolocation', value_vars=senior_high_data.columns [1:]) 

senior_high_data.rename(columns = {'value':'1.4.1p6 Net Enrolment Rate in secondary education (Senior High School)', 0 : 'Year'}, inplace=True)
senior_high_data = senior_high_data.astype({'Year':'int'})

Once that both datasets has been converted to their long representation, we can [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) the two datasets to the combined dataset based on the values of the `Geolocation` and the `Year` column with an outer join.

In [642]:
combined_data = combined_data.merge(junior_high_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.merge(senior_high_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [643]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School)
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN
...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN


#### 1.5.4. Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies
Then, the fourth dataset could be loaded using the same [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function.

In [644]:
data = pd.read_csv('data' + '/1.5.4.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/1.5.4.csv')
data

,1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies (Indicator can also found in SDG 13.1.3 and 11.b.2),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.0,2017,2018.0,2019,2020.0,2021.0,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,..,52.9,..,76.5,..,82.4,100.0,...
3,Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,..,94.0,..,97.5,..,79.5,61.5,...
4,Region I,..,..,..,..,..,..,..,..,..,...,..,..,..,44.8,..,100.0,..,74.4,76.7,...
5,Region II,..,..,..,..,..,..,..,..,..,...,..,..,..,100.0,..,100.0,..,49.0,55.1,...
6,Region III,..,..,..,..,..,..,..,..,..,...,..,..,..,59.0,..,99.3,..,100.0,100.0,...
7,Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,..,99.8,..,100.0,..,100.0,74.8,...
8,MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,..,82.0,..,100.0,..,100.0,100.0,...
9,Region V,..,..,..,..,..,..,..,..,..,...,..,..,..,91.0,..,93.3,..,57.5,56.7,...


Same as the previous datasets, we would need to [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the irrelevant rows at the bottom of the DataFrame. These are the rows that were a footer outside of the table in the csv files.

In [645]:
data = data.drop (data.index [19:])

Likewise, we know that the row at `Index 0` has the values that is the supposed column header for the table. However, checking each of the cells in this row would make us realize that the column header for the first column should not be `Year`, but rather `Geolocation` as the values in these columns refer to the different regions. 

Thus, we can change the value of the first column in this row to `Geolocation`, so that we would not need to rename the column if we directly made the 0th row into the column header. Then, we can [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the row at `Index 0` as it is now unnecessary. Additionally, we can see that there is a row of **NaN**s at `Index 1`, which would become the 0th row once we drop the row that became the column headers. This should be dropped also, before the index is resetted using the [`reset_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) function.

In [646]:
data.at[0, '1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies (Indicator can also found in SDG 13.1.3 and 11.b.2)'] = 'Geolocation'

In [647]:
data.columns = data.loc[0]
data = data.drop (data.index[0])

data = data.drop (data.index[0])
data = data.reset_index (drop=True)
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.0,2017,2018.0,2019,2020.0,2021.0,2022
0,National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,..,52.9,..,76.5,..,82.4,100.0,...
1,Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,..,94.0,..,97.5,..,79.5,61.5,...
2,Region I,..,..,..,..,..,..,..,..,..,...,..,..,..,44.8,..,100.0,..,74.4,76.7,...
3,Region II,..,..,..,..,..,..,..,..,..,...,..,..,..,100.0,..,100.0,..,49.0,55.1,...
4,Region III,..,..,..,..,..,..,..,..,..,...,..,..,..,59.0,..,99.3,..,100.0,100.0,...
5,Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,..,99.8,..,100.0,..,100.0,74.8,...
6,MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,..,82.0,..,100.0,..,100.0,100.0,...
7,Region V,..,..,..,..,..,..,..,..,..,...,..,..,..,91.0,..,93.3,..,57.5,56.7,...
8,Region VI,..,..,..,..,..,..,..,..,..,...,..,..,..,25.1,..,20.2,..,99.3,100.0,...
9,Region VII,..,..,..,..,..,..,..,..,..,...,..,..,..,100.0,..,87.5,..,94.1,100.0,...


The next step would be renaming the values under the `Geolocation`, although, as seen in the resulting table, we would notice that there is no row for **PHILIPPINES**. This is reflected in the way that we set the values of this column.

In [648]:
data ['Geolocation'] = region_names [1:]
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.0,2017,2018.0,2019,2020.0,2021.0,2022
0,NCR: National Capital Region,..,..,..,..,..,..,..,..,..,...,..,..,..,52.9,..,76.5,..,82.4,100.0,...
1,CAR: Cordillera Administrative Region,..,..,..,..,..,..,..,..,..,...,..,..,..,94.0,..,97.5,..,79.5,61.5,...
2,Region 1: Ilocos Region,..,..,..,..,..,..,..,..,..,...,..,..,..,44.8,..,100.0,..,74.4,76.7,...
3,Region 2: Cagayan Valley,..,..,..,..,..,..,..,..,..,...,..,..,..,100.0,..,100.0,..,49.0,55.1,...
4,Region 3: Central Luzon,..,..,..,..,..,..,..,..,..,...,..,..,..,59.0,..,99.3,..,100.0,100.0,...
5,Region 4A: CALABARZON,..,..,..,..,..,..,..,..,..,...,..,..,..,99.8,..,100.0,..,100.0,74.8,...
6,MIMAROPA: Southwestern Tagalog Region,..,..,..,..,..,..,..,..,..,...,..,..,..,82.0,..,100.0,..,100.0,100.0,...
7,Region 5: Bicol Region,..,..,..,..,..,..,..,..,..,...,..,..,..,91.0,..,93.3,..,57.5,56.7,...
8,Region 6: Western Visayas,..,..,..,..,..,..,..,..,..,...,..,..,..,25.1,..,20.2,..,99.3,100.0,...
9,Region 7: Central Visayas,..,..,..,..,..,..,..,..,..,...,..,..,..,100.0,..,87.5,..,94.1,100.0,...


As with the previous datasets, we would have to [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) the '..' and '...' values, which represents **null**, in the DataFrame with **NaN**s. This is to avoid any errors that would happen in these rows, and so that it would be represented properly.

In [649]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')

After all of this, we can now transform this dataset that is in its wide represetation into its long representation using the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function.

In [650]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 
data

,Geolocation,0,value
0,NCR: National Capital Region,2000,NaN
1,CAR: Cordillera Administrative Region,2000,NaN
2,Region 1: Ilocos Region,2000,NaN
3,Region 2: Cagayan Valley,2000,NaN
4,Region 3: Central Luzon,2000,NaN
...,...,...,...
386,Region 10: Northern Mindanao,2022,NaN
387,Region 11: Davao Region,2022,NaN
388,Region 12: SOCCSKSARGEN,2022,NaN
389,CARAGA: Cordillera Administrative Region,2022,NaN


Once we were able to convert it to its long representation, we would see that the column names in this new DataFrame are not descriptive with respect to the values underneath the column. Directly merging this with the combined DataFrame would make it hard for its users to distinguish what these columns are for, which is why it was [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)d to its correct column names.

In [651]:
data.rename(columns = {'value':'1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

After this, we can now [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) it to the combined dataframe.

In [652]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [653]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN
...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN


#### 3.4.1. Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease
To start with the fifth dataset, let us load the data from the csv file using pandas' [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function.

In [654]:
data = pd.read_csv('data' + '/3.4.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/3.4.1.csv')
data

,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,Year,NaN,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,2019,2020.0,2021,2022
1,Indicator,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.4.1 Mortality rate attributed to cardiovascu...,PHILIPPINES,Both Sexes,..,..,..,..,..,..,4.2,...,4.5,4.6,4.7,4.6,4.5,4.5,4.7,4.6,..,...
3,NaN,NaN,Male,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,5.6,..,...
4,NaN,NaN,Female,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,3.7,..,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,Note:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,.. - Data not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Based on the DataFrame that we got using the [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function, we can see that there are rows of **NaN**s at the lower part of the DataFrame. Upon further inspection, it started from `Index 266`, which is why the rows from this index was [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)ped.

In [655]:
data = data.drop (data.index [266:])

As the column headers are all **Unnamed**, we need to set the column headers to its correct value, which is found at `Index 0`. Although, the values for the first three columns in this row are not descriptive to be column headers, which is why we are changing their values to the correct descriptive name for the rows underneath them using the [`at`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) function.

As we have no use for the row at `Index 0`, we can now [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) this row. With this, we would also be [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)ping the next row as it is just a row of **NaN**s.

In [656]:
data.at[0, '3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease'] = 'Indicator'
data.at[0, 'Unnamed: 1'] = 'Geolocation'
data.at[0, 'Unnamed: 2'] = 'Sex'

In [657]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

As the `Sex` column is not available for all datasets, it was decided that only the total—or those rows with **Both Sexes**—would be considered. Once we our data only includes rows with **Both Sexes** as the value of their `Sex` column, we can now [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) this column as this column would only have one unique value.

In [658]:
data = data [data ['Sex'] == 'Both Sexes']
data = data.drop('Sex', axis = 1)
data = data.reset_index(drop=True)

Then, we need to [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) all cells that has the value of either '..' or '...' with **NaN** for better computation in the future. 

In [659]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')

Upon studying the different indicators under this specific Sustainable Development Goal (SDG), we would realize that it is comprised of different subsets: (1) cardiovascular diseases, (2) cancer, (3) diabetes, and (4) chronic respiratory disease. However, as we only aim to get the total mortality rate with respect to all of these diseases, we would only get the rows under this indicator which is from `Index 0` to `Index 16`.

Then, after dividing the different subsets, we can now [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the `Indicator` column. 

In [660]:
data['Indicator'].unique()

array(['3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease',
       nan,
       '..3.4.1.1 Mortality rate attributed to cardiovascular disease',
       '..3.4.1.2 Mortality rate attributed to cancer',
       '..3.4.1.3 Mortality rate attributed to diabetes',
       '..3.4.1.4 Mortality rate attributed to chronic respiratory disease'],
      dtype=object)

In [661]:
all_data = data [0:16]
cardio_data = data [16:34]
cancer_data = data [34:52]
diabetes_data = data [52:70]
respi_data = data [70:]

In [662]:
all_data = all_data.drop('Indicator', axis = 1)
all_data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020.0,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,4.2,4.2,4.3,...,4.5,4.6,4.7,4.6,4.5,4.5,4.7,4.6,NaN,NaN
1,..National Capital Region (NCR),NaN,NaN,NaN,NaN,NaN,NaN,5.1,5.2,5.2,...,5.2,5.3,5.5,5.2,4.9,4.9,5,4.8,NaN,NaN
2,..Cordillera Administrative Region (CAR),NaN,NaN,NaN,NaN,NaN,NaN,3.3,3.1,3.3,...,3.4,3.5,3.7,3.6,3.6,3.8,4.1,3.8,NaN,NaN
3,..Region I,NaN,NaN,NaN,NaN,NaN,NaN,4.9,4.8,5,...,5,5.1,5.1,5,4.9,4.9,4.9,4.9,NaN,NaN
4,..Region II,NaN,NaN,NaN,NaN,NaN,NaN,4,3.9,4,...,4.4,4.4,4.5,4.4,4.3,4.5,4.7,4.3,NaN,NaN
5,..Region III,NaN,NaN,NaN,NaN,NaN,NaN,4.8,5,5,...,5.2,5.4,5.4,5.3,5.2,5.2,5.3,5.2,NaN,NaN
6,..Region IV-A,NaN,NaN,NaN,NaN,NaN,NaN,4.7,4.7,4.6,...,4.9,5.1,5,4.9,4.9,4.9,5.1,5.1,NaN,NaN
7,..MIMAROPA,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.5,3.5,...,3.8,3.9,3.9,4.2,3.9,4.1,4.3,4.2,NaN,NaN
8,..Region VII,NaN,NaN,NaN,NaN,NaN,NaN,4.3,4.2,4.4,...,4.7,4.8,5,4.9,4.7,4.7,4.8,4.9,NaN,NaN
9,..Region VIII,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.5,3.5,...,3.7,3.8,3.8,3.7,3.7,3.8,3.9,4.0,NaN,NaN


Upon inspection, we would realize that there are two regions that are missing from the table, which are **Region V** and **Region VI**, which is why we would only be using the region names that are included in the DataFrame. 

In [663]:
# no region five and six
all_data ['Geolocation'] = region_names [0:8] + region_names [10:]

After this, with the use of the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function, we can now convert our DataFrame to its long representation. Then, we must set the column headers to describe the values in this column, which is why we would need to [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) the columns. 

In [664]:
all_data = pd.melt(all_data, id_vars='Geolocation', value_vars=all_data.columns [1:]) 

all_data.rename(columns = {'value':'3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)', 0 : 'Year'}, inplace=True)
all_data = all_data.astype({'Year':'int'})

After this, we can now [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) it to the DataFrame which holds the combined datasets.

In [665]:
combined_data = combined_data.merge(all_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [666]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)"
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.7.1. Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods

Using the same [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function, we load the sixth dataset. 

In [667]:
data = pd.read_csv('data' + '/3.7.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/3.7.1.csv')
data

,3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Year,NaN,2000,2001,2002,2003.0,2004,2005,2006,2007,...,2013.0,2014,2015,2016,2017.0,2018,2019,2020,2021,2022
1,Indicator/Sub-indicators,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.7.1 Proportion of women of reproductive age ...,PHILIPPINES,..,..,..,46.7,..,..,..,..,...,51.8,..,..,..,56.9,..,..,..,..,...
3,NaN,..National Capital Region (NCR),..,..,..,47.2,..,..,..,..,...,53.4,..,..,..,59.5,..,..,..,..,...
4,NaN,..Cordillera Administrative Region (CAR),..,..,..,44.4,..,..,..,..,...,59.8,..,..,..,66.7,..,..,..,..,...
5,NaN,..Region I,..,..,..,49.6,..,..,..,..,...,50.8,..,..,..,59.5,..,..,..,..,...
6,NaN,..Region II,..,..,..,68.8,..,..,..,..,...,69.1,..,..,..,74.1,..,..,..,..,...
7,NaN,..Region III,..,..,..,54.2,..,..,..,..,...,60.4,..,..,..,56.8,..,..,..,..,...
8,NaN,..Region IV-A,..,..,..,46.1,..,..,..,..,...,49.1,..,..,..,49.2,..,..,..,..,...
9,NaN,..MIMAROPA,..,..,..,48.5,..,..,..,..,...,55.1,..,..,..,61.7,..,..,..,..,...


Irrelevant rows that are just footers for the file are also [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)ped. From the DataFrame above, we can see that these are the rows from `Index 20`.

In [668]:
data = data.drop (data.index [20:])

Additionally, we can see that the current column names are **Unnamed**. Thus, we have to set the column names to its correct values so that we can determine what the values in the columns are.

Understanding the data, we can see that the row at `Index 0` holds the value for the column headers. However, there is a **NaN** value, which should be **Geolocation** based on the data underneath it. This is why the value of this cell was changed to **Geolocation** using the [`at`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) function.

This is done before the column names was set to the row at `Index 0`, and then [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)ping this row and the row of NaNs at the next row.

In [669]:
data.at[0, 'Unnamed: 1'] = 'Geolocation'

In [670]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)
data

,Year,Geolocation,2000,2001,2002,2003.0,2004,2005,2006,2007,...,2013.0,2014,2015,2016,2017.0,2018,2019,2020,2021,2022
0,3.7.1 Proportion of women of reproductive age ...,PHILIPPINES,..,..,..,46.7,..,..,..,..,...,51.8,..,..,..,56.9,..,..,..,..,...
1,NaN,..National Capital Region (NCR),..,..,..,47.2,..,..,..,..,...,53.4,..,..,..,59.5,..,..,..,..,...
2,NaN,..Cordillera Administrative Region (CAR),..,..,..,44.4,..,..,..,..,...,59.8,..,..,..,66.7,..,..,..,..,...
3,NaN,..Region I,..,..,..,49.6,..,..,..,..,...,50.8,..,..,..,59.5,..,..,..,..,...
4,NaN,..Region II,..,..,..,68.8,..,..,..,..,...,69.1,..,..,..,74.1,..,..,..,..,...
5,NaN,..Region III,..,..,..,54.2,..,..,..,..,...,60.4,..,..,..,56.8,..,..,..,..,...
6,NaN,..Region IV-A,..,..,..,46.1,..,..,..,..,...,49.1,..,..,..,49.2,..,..,..,..,...
7,NaN,..MIMAROPA,..,..,..,48.5,..,..,..,..,...,55.1,..,..,..,61.7,..,..,..,..,...
8,NaN,..Region V,..,..,..,30.6,..,..,..,..,...,29.3,..,..,..,44.4,..,..,..,..,...
9,NaN,..Region VI,..,..,..,42.3,..,..,..,..,...,45.5,..,..,..,56.8,..,..,..,..,...


Added to this, we can see that there is a column of **NaN**s, which we do not need, so we can also [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) this.

In [671]:
data = data.drop('Year', axis=1)

Just like what we have done in the previous datasets, we would rename the **Geolocation** column based on the common names of the region for easier understanding of the dataset.

In [672]:
data ['Geolocation'] = region_names

As the missing data or null values in the dataset are represented by '..' or '...', which are strings that might affect the computations that might be done in this numerical columns, we would be using the [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) function to replace these string values to **np.nan**.

In [673]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')
data

,Geolocation,2000,2001,2002,2003.0,2004,2005,2006,2007,2008.0,...,2013.0,2014,2015,2016,2017.0,2018,2019,2020,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,46.7,NaN,NaN,NaN,NaN,46.8,...,51.8,NaN,NaN,NaN,56.9,NaN,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,47.2,NaN,NaN,NaN,NaN,43.2,...,53.4,NaN,NaN,NaN,59.5,NaN,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,44.4,NaN,NaN,NaN,NaN,55.0,...,59.8,NaN,NaN,NaN,66.7,NaN,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,49.6,NaN,NaN,NaN,NaN,49.7,...,50.8,NaN,NaN,NaN,59.5,NaN,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,68.8,NaN,NaN,NaN,NaN,62.6,...,69.1,NaN,NaN,NaN,74.1,NaN,NaN,NaN,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,54.2,NaN,NaN,NaN,NaN,54.0,...,60.4,NaN,NaN,NaN,56.8,NaN,NaN,NaN,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,46.1,NaN,NaN,NaN,NaN,46.1,...,49.1,NaN,NaN,NaN,49.2,NaN,NaN,NaN,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,48.5,NaN,NaN,NaN,NaN,48.5,...,55.1,NaN,NaN,NaN,61.7,NaN,NaN,NaN,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,30.6,NaN,NaN,NaN,NaN,33.8,...,29.3,NaN,NaN,NaN,44.4,NaN,NaN,NaN,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,42.3,NaN,NaN,NaN,NaN,44.4,...,45.5,NaN,NaN,NaN,56.8,NaN,NaN,NaN,NaN,NaN


As the dataset now looks like the wide representation that we wanted, we would be transforming it to its long representation, using the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function, so that we could merge it to the combined dataset.

In [674]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 
data

,Geolocation,0,value
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


Although, before merging it to the combined dataset, we would need to [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) the columns `0` and `value`, as they are not descriptive enough. If we directly merged it to the combined dataset, we might not be able to determine what the values in these columns mean. 

In [675]:
data.rename(columns = {'value':'3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})
data

,Geolocation,Year,3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


Once the column names have been fixed, we could use the [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function to use outer join to merge the two datasets.

In [676]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [677]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.7.2. Adolescent birth rate aged 15-19 years per 1,000 women in that age group
Then, the seventh dataset could be loaded using the same [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function.

In [678]:
data = pd.read_csv('data' + '/3.7.2.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/3.7.2.csv')
data

,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003.0,2004,2005,2006,2007,2008.0,...,2013.0,2014,2015,2016,2017.0,2018,2019,2020,2021,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,..,..,53.0,..,..,..,..,54.0,...,57.0,..,..,..,47.0,..,..,..,..,...
3,..National Capital Region (NCR),..,..,..,35.0,..,..,..,..,25.0,...,48.0,..,..,..,27.0,..,..,..,..,...
4,..Cordillera Administrative Region (CAR),..,..,..,52.0,..,..,..,..,34.0,...,53.0,..,..,..,25.0,..,..,..,..,...
5,..Region I,..,..,..,55.0,..,..,..,..,52.0,...,78.0,..,..,..,46.0,..,..,..,..,...
6,..Region II,..,..,..,85.0,..,..,..,..,54.0,...,65.0,..,..,..,51.0,..,..,..,..,...
7,..Region III,..,..,..,42.0,..,..,..,..,69.0,...,63.0,..,..,..,61.0,..,..,..,..,...
8,..Region IV-A,..,..,..,44.0,..,..,..,..,63.0,...,58.0,..,..,..,37.0,..,..,..,..,...
9,..MIMAROPA,..,..,..,108.0,..,..,..,..,87.0,...,68.0,..,..,..,47.0,..,..,..,..,...


As seen in the previous datasets, there are three types of columns that are processed and [`drop`]ped first: (1) the irrelevant rows that were footers in the .csv file, (2) the row that would be turned into the column headers, and (3) the row of **NaN**s.

In [679]:
data = data.drop (data.index [20:])

In [680]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)
data

,Year,2000,2001,2002,2003.0,2004,2005,2006,2007,2008.0,...,2013.0,2014,2015,2016,2017.0,2018,2019,2020,2021,2022
0,PHILIPPINES,..,..,..,53.0,..,..,..,..,54.0,...,57.0,..,..,..,47.0,..,..,..,..,...
1,..National Capital Region (NCR),..,..,..,35.0,..,..,..,..,25.0,...,48.0,..,..,..,27.0,..,..,..,..,...
2,..Cordillera Administrative Region (CAR),..,..,..,52.0,..,..,..,..,34.0,...,53.0,..,..,..,25.0,..,..,..,..,...
3,..Region I,..,..,..,55.0,..,..,..,..,52.0,...,78.0,..,..,..,46.0,..,..,..,..,...
4,..Region II,..,..,..,85.0,..,..,..,..,54.0,...,65.0,..,..,..,51.0,..,..,..,..,...
5,..Region III,..,..,..,42.0,..,..,..,..,69.0,...,63.0,..,..,..,61.0,..,..,..,..,...
6,..Region IV-A,..,..,..,44.0,..,..,..,..,63.0,...,58.0,..,..,..,37.0,..,..,..,..,...
7,..MIMAROPA,..,..,..,108.0,..,..,..,..,87.0,...,68.0,..,..,..,47.0,..,..,..,..,...
8,..Region V,..,..,..,60.0,..,..,..,..,63.0,...,59.0,..,..,..,36.0,..,..,..,..,...
9,..Region VI,..,..,..,57.0,..,..,..,..,41.0,...,58.0,..,..,..,38.0,..,..,..,..,...


Although, we can see that there is a column name that does not correctly represent the data of this column: the `Year` column does not indicate years, but rather the regions. This is why it was [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)d to `Geolocation`. 

In [681]:
data.rename(columns = {'Year':'Geolocation'}, inplace=True)

Once we have cleaned the column headers, the values for the `Geolocation` column would be fixed to include their common names. It is important to note that it was made sure that each of the row completely match the arrangement in the `region_name` variable.

In [682]:
data ['Geolocation'] = region_names

As we now have fixed the number of rows and the column names, we would now replace the string representation of null or missing vlaues. This is done with the use of [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) function, which would convert the '..' and '...' values into **np.nan**.

In [683]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')

Then, we can now convert our DataFrame into its long representation using the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function. As in the processing of the previous datasets, we would have to [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) the column names as they are not descriptive enough.

In [684]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

As we are now sure that the missing or null values are correctly represented, the values of the `Geolocation` are now more easily understandable, and the column headers are descriptive enough, we can now merge this dataset into the combined datasets using the [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function.

In [685]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [686]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group"
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 4.1.s1. Completion Rate of elementary and secondary students
To start with the eighth dataset, let us load the data from the csv file using pandas' [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function.

In [687]:
data = pd.read_csv('data' + '/4.1.s1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/4.1.s1.csv')
data

,4.1.s1 Completion Rate of elementary and secondary students 1/ 2/,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,Year,NaN,NaN,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018.00,2019.00,2020.000000,2021,2022
1,Geolocation,Level of Education,Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,Elementary,Both Sexes,62.72,68.18,71.55,70.24,69.06,68.11,71.72,...,77.67,83.74,84.02,93.06,92.41,97.15,96.56,82.510000,...,...
3,NaN,NaN,Female,65.53,70.7,76.32,75.63,75.2,73.46,76.7,...,81.33,86.23,87.43,95.52,94.61,99.12,98.08,84.681828,...,...
4,NaN,NaN,Male,60.05,65.78,67.23,65.42,63.63,63.29,67.28,...,74.38,81.45,80.97,90.83,90.41,95.26,95.10,80.500538,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,.. - Data not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,... - Data not yet available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,1/ - Updates were based on the submission of D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,2/ - Estimation in Senior High School only sta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From the view of the DataFrame above, we can see that there are unnecessary rows captured by the  [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function. To be able to correctly represent the data, we would need to [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) these rows.

In [688]:
data = data.drop(data.index[164:])

Another problem that we have based on the DataFrame shown above is the lack of column names, as shown in the **Unnamed** values in the header. Studying the DataFrame, we would find the supposed column headers in the row of `Index 0`, though we face the problem of having **NaN** values at the first three columns of this row. This is why the values in these cells are changed using the [`at`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html) function, before converting this row to be the column header.

After we have been able to turn this into the column header, we would need to drop this row and the row beneath it as they are unnecessary rows.

In [689]:
data.at[0, '4.1.s1 Completion Rate of elementary and secondary students 1/ 2/'] = 'Geolocation'
data.at[0, 'Unnamed: 1'] = 'Level of Education'
data.at[0, 'Unnamed: 2'] = 'Sex'

In [690]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

Just like in datasets that has the `Sex` column, we would only be getting rows with the value for this column as **Both Sexes**. Afterwards, as we already have no need for this column anymore, we can [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) it. 

In [691]:
data = data [data['Sex'] == 'Both Sexes']
data = data.drop ('Sex', axis = 1)
data = data.reset_index(drop=True)
data

,Geolocation,Level of Education,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018.0,2019.0,2020.0,2021,2022
0,PHILIPPINES,Elementary,62.72,68.18,71.55,70.24,69.06,68.11,71.72,73.06,...,77.67,83.74,84.02,93.06,92.41,97.15,96.56,82.510000,...,...
1,NaN,Secondary (Junior High School),70.07,69.97,74.81,71.67,72.38,61.66,72.14,75.37,...,76.25,77.77,74.03,80.91,84.32,88.84,85.75,82.111684,...,...
2,NaN,Secondary (Senior High School),..,..,..,..,..,..,..,..,...,..,..,..,..,..,81.01,76.71,69.317762,...,...
3,..National Capital Region (NCR),Elementary,63.87,74.29,84.35,83.81,82.1,82.5,88.48,85.35,...,78.72,74.71,82.29,85.97,94.65,99.04,94.97,69.360000,...,...
4,NaN,Secondary (Junior High School),68.16,68.43,75.51,73.36,77.33,65.87,71.62,78.71,...,76.33,77.27,74.23,80.29,90.62,92.80,87.31,73.645274,...,...
5,NaN,Secondary (Senior High School),..,..,..,..,..,..,..,..,...,..,..,..,..,..,82.64,76.28,56.255397,...,...
6,..Cordillera Administrative Region (CAR),Elementary,61.95,59.55,77.61,73.9,69.46,67.53,74.99,71.7,...,82.61,86.43,86.22,93.51,91.64,97.52,95.81,94.560000,...,...
7,NaN,Secondary (Junior High School),70.31,61.75,59.41,73.61,72.54,63.2,83.69,75.67,...,76.34,76.91,69.97,79.78,81.23,87.01,81.69,87.860475,...,...
8,NaN,Secondary (Senior High School),..,..,..,..,..,..,..,..,...,..,..,..,..,..,81.07,76.25,81.151454,...,...
9,..Region I,Elementary,78.73,79.7,86.74,84.46,85.49,85.48,81.64,82.71,...,91.14,94.54,91.5,97.41,95.45,98.10,98.97,97.320000,...,...


As we can see from the resulting dataset, there are still **NaN** values in the `Geolocation` column, which we do not want as this would be used in merging the datasets together. However, if we study it, we would realize that the reason for this is that one value for `Geolocation` actually spans to the next two rows as there are different values for the `Level of Education` column. Although, we cannot just separate the dataset per unique value of the `Level of Education` column, as the `Geolocation` would be NaN for all  **Secondary (Junior High School)** and **Secondary (Senior High School)**. 

Due to this, we copy the value of the `Geolocation` column of a row to the next two rows after it. 

In [692]:
# copying the geolocation value to the next two rows
i = 0
while i < len (data):
    if i % 3 == 0:
        data.at[i + 1, 'Geolocation'] = data['Geolocation'][i]
        data.at[i + 2, 'Geolocation'] = data['Geolocation'][i]
        i = i + 3

Before we divide the dataset based on the value of `Level of Education`, we must first replace cells with the strings '..' or '...' with **np.nan**. This is so that we would not need to process this representation of missing or null values separately (i.e., per division). Then, we can now separate them so that we can properly label it before merging it to the combined dataset.

In [693]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')

In [694]:
elem_data = data [data['Level of Education'] == 'Elementary']
elem_data = elem_data.reset_index (drop=True)

junior_data = data [data['Level of Education'] == 'Secondary (Junior High School)']
junior_data = junior_data.reset_index (drop=True)

senior_data = data [data['Level of Education'] == 'Secondary (Senior High School)']
senior_data = senior_data.reset_index (drop=True)

Once we have successfully divided the dataset based on the value of the `Level of Education` column, we can now [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) this column as each of the division would technically only have one value for this column.

In [695]:
elem_data = elem_data.drop ('Level of Education', axis = 1)
elem_data = elem_data.reset_index(drop=True)

In [696]:
junior_data = junior_data.drop ('Level of Education', axis = 1)
junior_data = junior_data.reset_index(drop=True)

In [697]:
senior_data = senior_data.drop ('Level of Education', axis = 1)
senior_data = senior_data.reset_index(drop=True)

After making sure that the arrangement of the region matches the arrangement of the values of the `region_names` variable, we can change the values of the `Geolocation` column for each of the division. 

In [698]:
elem_data ['Geolocation'] = region_names

In [699]:
junior_data ['Geolocation'] = region_names

In [700]:
senior_data ['Geolocation'] = region_names

Then, we can now convert the DataFrames into their long representation, before using the [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) function to make the column names more descriptive of the data in the columns.

In [701]:
elem_data = pd.melt(elem_data, id_vars='Geolocation', value_vars=elem_data.columns [1:]) 

elem_data.rename(columns = {'value':'4.1.s1 Completion Rate of elementary and secondary students (Elementary)', 0 : 'Year'}, inplace=True)
elem_data = elem_data.astype({'Year':'int'})

In [702]:
junior_data = pd.melt(junior_data, id_vars='Geolocation', value_vars=junior_data.columns [1:]) 

junior_data.rename(columns = {'value':'4.1.s1 Completion Rate of elementary and secondary students (Junior High School)', 0 : 'Year'}, inplace=True)
junior_data = junior_data.astype({'Year':'int'})

In [703]:
senior_data = pd.melt(senior_data, id_vars='Geolocation', value_vars=senior_data.columns [1:]) 

senior_data.rename(columns = {'value':'4.1.s1 Completion Rate of elementary and secondary students (Senior High School)', 0 : 'Year'}, inplace=True)
senior_data = senior_data.astype({'Year':'int'})

As we have now made sure that each of division would be understandable even if combined with the combined dataset, we can now [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) each of them into the combined dataset

In [704]:
combined_data = combined_data.merge(elem_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [705]:
combined_data = combined_data.merge(junior_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [706]:
combined_data = combined_data.merge(senior_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [707]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",4.1.s1 Completion Rate of elementary and secondary students (Elementary),4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School)
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,62.72,70.07,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,63.87,68.16,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,61.95,70.31,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,78.73,73.38,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,70.75,72.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 4.c.s2. Number of Technical-Vocational Education and Training (TVET) trainers trained
Next, we can load the ninth dataset.

In [708]:
data = pd.read_csv('data' + '/4.c.s2.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/4.c.s2.csv')
data

,4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,..,..,..,..,..,..,..,..,...,..,..,..,6518.0,11159.0,10118.0,10855.0,4023.0,7746.0,...
3,..National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,..,610.0,1028.0,1280.0,1409.0,782.0,1985.0,...
4,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,..,201.0,302.0,166.0,260.0,92.0,199.0,...
5,..Region I,..,..,..,..,..,..,..,..,..,...,..,..,..,474.0,455.0,475.0,501.0,375.0,327.0,...
6,..Region II,..,..,..,..,..,..,..,..,..,...,..,..,..,270.0,612.0,447.0,686.0,215.0,240.0,...
7,..Region III,..,..,..,..,..,..,..,..,..,...,..,..,..,280.0,262.0,354.0,839.0,277.0,471.0,...
8,..Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,..,833.0,1067.0,1440.0,817.0,177.0,647.0,...
9,..MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,..,139.0,523.0,709.0,413.0,162.0,255.0,...


As usual, we would first be [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)ping the irrelevant rows. 

In [709]:
data = data.drop(data.index[20:])

Then, as we know that the correct column headers are found at `Index 0`, we have to fix the values of this row to fully represent the data in the columns. This is why the **Year** value was changed into **Geolocation** because the values in this column are the rows of the country.

After this, we can now make the value of this row as the value of the column headers, before [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)ping this row as it would not be used anymore. In line with this, we can also [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) the row of **NaN**s underneath this row.

In [710]:
data.at[0, '4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained'] = 'Geolocation'

In [711]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

Then, we need to change the values of the `Geolocation` column to match the prescribed format for the region names.

In [712]:
data ['Geolocation'] = region_names

After this, we need to clean the dataset by turning the string representation of missing or null values, which are '..' and '...', into **np.nan**. This would allow us to correctly use mathematical functions into these columns without errors arising due to strings.

In [713]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

# data = data.dropna(axis=1, how = 'all')

Once we have done this, we can convert the DataFrame into its long representation, which would allow us to merge it with the combined dataset. Converting a DataFrame that is in its wide representation into its long representation is made possible by the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function.

However, using the [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) function would result into a three-column DataFrame which has the following column names: (1) `Geolocation`, (2) `0`, and (3) `value`. The last two columns are not properly descriptive of the values of the column, which is why these two columns are [`rename`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)d. 

In [714]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

As we now have a DataFrame that is in its long representation, we can now [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) it to the combined DataFrame, with respect to the values of the `Geolocation` and `Year` columns. This means that a row from this DataFrame would be [`merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)d into the combined dataset on the row that has the same `Geolocation` and `Year`. 

In [715]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [716]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",4.1.s1 Completion Rate of elementary and secondary students (Elementary),4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,62.72,70.07,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,63.87,68.16,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,61.95,70.31,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,78.73,73.38,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,70.75,72.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 7.1.1. Proportion of population with access to electricity

Now, we will proceed to loading the tenth dataset.

In [717]:
data = pd.read_csv('data' + '/7.1.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/7.1.1.csv') // AJ TO DO
data

,7.1.1 Proportion of population with access to electricity 1/,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2020.00,2021,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,..,..,..,..,..,..,..,..,...,79.853466,80.900000,89.620000,90.650000,91.090000,96.120000,92.960000,94.49,...,...
3,..National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,97.737787,98.027099,98.023644,98.259443,103.023194,107.614820,100.000000,100.00,...,...
4,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,78.061518,83.482667,87.491115,90.300000,92.451772,95.622169,93.319710,94.30,...,...
5,..Region I,..,..,..,..,..,..,..,..,..,...,83.917453,86.436310,93.663932,93.900447,99.154068,102.201664,98.687338,99.02,...,...
6,..Region II,..,..,..,..,..,..,..,..,..,...,81.122927,83.534027,92.294218,93.095178,94.770382,97.913797,95.535692,99.63,...,...
7,..Region III,..,..,..,..,..,..,..,..,..,...,93.178333,92.299013,96.871581,97.373164,107.082482,109.124555,99.429197,99.74,...,...
8,..Region IV-A,..,..,..,..,..,..,..,..,..,...,93.744276,92.033842,95.396047,96.670558,104.925142,110.433491,99.012440,99.17,...,...
9,..MIMAROPA,..,..,..,..,..,..,..,..,..,...,67.231577,69.355541,82.033903,84.141455,82.919516,87.270240,90.250458,91.01,...,...


Before anything else, we drop the irrelevant rows.

In [718]:
data = data.drop(data.index[20:])

First, we will change the data in Index 0 at column '7.1.1 Proportion of population with access to electricity 1/' into 'Geolocation' since our goal is to make the geolocation the first column of the dataframe. By doing this, Index 0 has now the correct column headers. 

With this, we have to arrange the values of this row to fully represent the data in the columns. Therefeore, we will now make the value of this row as the value of the column headers. 

After this, we drop this row (Index 0) as it would not be used anymore as well as the row of NaNs underneath this row.

In [719]:
data.at[0,'7.1.1 Proportion of population with access to electricity 1/'] = 'Geolocation'

In [720]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

After checking if the order of the Geolocation is the same as what we intended, we will initialize the Geolocation column of the region names to make sure that the format of the region names in this dataset is the same as the currently combined dataset.

In [721]:
data ['Geolocation'] = region_names

We will then change the the '..' or '...' strings to NaN using the np.nan. Again, these NaN values were not dropped because all years from 2001-2022 will be in the combined dataset.

In [722]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

We can now convert the DataFrame into its long representation using the `melt` function. This would allow us to merge it with the combined dataset.

In [723]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'7.1.1 Proportion of population with access to electricity', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

In [724]:
data

,Geolocation,Year,7.1.1 Proportion of population with access to electricity
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


We will now combine this dataset to the currently combined dataset.

In [725]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [726]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",4.1.s1 Completion Rate of elementary and secondary students (Elementary),4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,62.72,70.07,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,63.87,68.16,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,61.95,70.31,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,78.73,73.38,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,70.75,72.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 8.1.1. Annual growth rate of real GDP per capita

Loading the eleventh dataset...

In [727]:
data = pd.read_csv('data' + '/8.1.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/8.1.1.csv')
data

,8.1.1 Annual growth rate of real GDP per capita,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001.000000,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,...,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2020.000000,2021,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,1.007914,1.691458,3.064526,4.541527,2.978552,3.372089,4.578884,2.463484,...,4.948185,4.573827,4.594970,5.376185,5.248500,4.746743,4.602268,-10.806602,4.2,...
3,..National Capital Region (NCR),..,0.841128,-0.916195,2.756815,6.355851,3.310895,3.809951,4.989652,2.771026,...,4.912506,3.842172,4.685639,5.511500,4.014293,4.087414,5.634448,-11.200476,...,...
4,..Cordillera Administrative Region (CAR),..,1.900838,2.604040,3.268534,3.487977,0.056740,2.068305,5.577005,2.103372,...,5.453843,3.919264,3.104855,1.624092,8.263996,4.611166,3.393800,-10.682230,...,...
5,..Region I,..,0.811589,1.704884,3.183010,3.889663,3.362599,4.594092,4.671401,1.774523,...,7.382397,5.465228,4.289072,6.919629,4.497287,4.914167,6.305168,-8.477176,...,...
6,..Region II,..,2.147953,0.089252,1.132812,7.426633,-3.238956,8.370662,5.103214,1.083639,...,7.554134,7.074622,2.846142,3.468013,6.478263,3.337775,5.689419,-10.815141,...,...
7,..Region III,..,3.600192,3.351012,2.107589,0.962747,1.725320,2.538984,3.720296,3.033030,...,3.967256,5.702767,4.082271,5.682582,8.215194,4.902421,4.003513,-15.362293,...,...
8,..Region IV-A,..,-1.683998,1.524221,1.948337,2.276289,2.138433,1.490928,2.573883,0.684335,...,4.635088,3.177655,4.289997,4.283138,5.252561,4.777452,2.452046,-12.218949,...,...
9,..MIMAROPA,..,1.664022,12.480291,7.750968,1.830638,7.306973,2.090920,7.619860,3.256205,...,2.722557,8.496021,3.320929,3.334896,4.766650,7.052686,2.957077,-8.745932,...,...


We will now drop the irrelevant rows.

In [728]:
data = data.drop(data.index[20:])

Observing the header column and the Index 0, the data in the Index 0 is much more similar to the column names we want for the dataset. With this, it would be more hassle to change all columns names in the header column than changing the data in Index 0 and setting it to be the header column.

With this, we will change the data in at Index 0 Column 0 into 'Geolocation' since our goal is to make the geolocation the first column of the dataframe. By doing this, Index 0 has now the correct column headers. Then, will now make the value of this row as the value of the column headers.

After this, we drop this row (Index 0) as it would not be used anymore as well as the row of NaNs underneath this.

In [729]:
data.at[0,'8.1.1 Annual growth rate of real GDP per capita'] = 'Geolocation'
data.head()

,8.1.1 Annual growth rate of real GDP per capita,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Geolocation,2000,2001.000000,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,...,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2020.000000,2021,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,1.007914,1.691458,3.064526,4.541527,2.978552,3.372089,4.578884,2.463484,...,4.948185,4.573827,4.594970,5.376185,5.248500,4.746743,4.602268,-10.806602,4.2,...
3,..National Capital Region (NCR),..,0.841128,-0.916195,2.756815,6.355851,3.310895,3.809951,4.989652,2.771026,...,4.912506,3.842172,4.685639,5.511500,4.014293,4.087414,5.634448,-11.200476,...,...
4,..Cordillera Administrative Region (CAR),..,1.900838,2.604040,3.268534,3.487977,0.056740,2.068305,5.577005,2.103372,...,5.453843,3.919264,3.104855,1.624092,8.263996,4.611166,3.393800,-10.682230,...,...


In [730]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

After checking if the order of the Geolocation is the same as what we intended, we will initialize the Geolocation column of the region names.

In [731]:
data ['Geolocation'] = region_names

To represent the missing values clearly, we change the the '..' or '...' strings to NaN using the np.nan.

In [732]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

We can now convert the DataFrame into its long representation to allow us to merge it with the combined dataset.

In [733]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'8.1.1 Annual growth rate of real GDP per capita', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

In [734]:
data[200:]

,Geolocation,Year,8.1.1 Annual growth rate of real GDP per capita
200,CAR: Cordillera Administrative Region,2011,-0.818431
201,Region 1: Ilocos Region,2011,2.760202
202,Region 2: Cagayan Valley,2011,5.009277
203,Region 3: Central Luzon,2011,5.734126
204,Region 4A: CALABARZON,2011,-0.346203
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


After this, we combine this dataset with the currently combined dataset.

In [735]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [736]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",4.1.s1 Completion Rate of elementary and secondary students (Elementary),4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,62.72,70.07,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,63.87,68.16,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,61.95,70.31,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,78.73,73.38,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,70.75,72.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 10.1.1. Growth rates of household expenditure or income per capita among the bottom 40 per cent of the population and the total population

Loading the twelfth dataset...

In [737]:
data = pd.read_csv('data' + '/10.1.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/10.1.1.csv') 
data

,10.1.1 Growth rates of household expenditure or income per capita among the bottom 40 per cent of the population and the total population,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015.000000,2016,2017,2018.000000,2019,2020,2021,2022
1,Indicator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.1.1.1 Bottom 40 percent of the population,..,..,..,..,..,..,..,..,..,...,..,..,7.406738,..,..,9.261018,..,..,...,...
3,..National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,5.883486,..,..,7.085782,..,..,...,...
4,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,6.635383,..,..,9.647948,..,..,...,...
5,..Region I,..,..,..,..,..,..,..,..,..,...,..,..,7.453736,..,..,11.312069,..,..,...,...
6,..Region II,..,..,..,..,..,..,..,..,..,...,..,..,8.900391,..,..,4.499523,..,..,...,...
7,..Region III,..,..,..,..,..,..,..,..,..,...,..,..,6.519357,..,..,8.215452,..,..,...,...
8,..Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,6.307004,..,..,9.511903,..,..,...,...
9,..MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,9.530952,..,..,10.132831,..,..,...,...


Dropping the irrelevant rows...

In [738]:
data = data.drop(data.index[38:])

We will change the data in at Index 0 Column 0 into 'Geolocation' since our goal is to make the geolocation the first column of the dataframe. By doing this, Index 0 has now the correct column headers. Then, will now make the value of this row as the value of the column headers.

After this, we drop this row (Index 0) as it would not be used anymore as well as the row of NaNs underneath this.

In [739]:
data.at[0,'10.1.1 Growth rates of household expenditure or income per capita among the bottom 40 per cent of the population and the total population'] = 'Geolocation'

In [740]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

To represent the missing values clearly, we change the the '..' or '...' strings to NaN using the np.nan.

In [741]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

As observed in this dataset, we have two parts which are **10.1.1.1 Bottom 40 percent of the population** and **10.1.1.2 Total Population**. Since we will both need these parts, we will still get both parts to combine with other datasets. However, we will divide them into two different datasets.

In [742]:
data['Geolocation'].unique()

array(['10.1.1.1 Bottom 40 percent of the population',
       '..National Capital Region (NCR)',
       '..Cordillera Administrative Region (CAR)', '..Region I',
       '..Region II', '..Region III', '..Region IV-A', '..MIMAROPA',
       '..Region V', '..Region VI', '..Region VII', '..Region VIII',
       '..Region IX', '..Region X', '..Region XI', '..Region XII',
       '..Caraga', '..BARMM', '10.1.1.2 Total Population'], dtype=object)

**10.1.1.1 Bottom 40 percent of the population** goes to `bottom_popu_data` while **10.1.1.2 Total Population** goes to `total_popu_data`.

In [743]:
bottom_popu_data = data [0:18]
total_popu_data = data [18:]

Since `total_popu_data` started with index 18, we will set its starting index to 0. 

Also, since the first row of each of the parts is a record for the Philippines and the order of the geolocation of each DataFrame is correct, we will initialize it with the region_names for uniformity.

In [744]:
total_popu_data = total_popu_data.reset_index (drop=True)

bottom_popu_data ['Geolocation'] = region_names
total_popu_data ['Geolocation'] = region_names

<ipython-input-744-8def1e9c97b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bottom_popu_data ['Geolocation'] = region_names


In [745]:
bottom_popu_data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015.0,2016,2017,2018.0,2019,2020,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.406738,NaN,NaN,9.261018,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.883486,NaN,NaN,7.085782,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.635383,NaN,NaN,9.647948,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.453736,NaN,NaN,11.312069,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.900391,NaN,NaN,4.499523,NaN,NaN,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.519357,NaN,NaN,8.215452,NaN,NaN,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6.307004,NaN,NaN,9.511903,NaN,NaN,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.530952,NaN,NaN,10.132831,NaN,NaN,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.240401,NaN,NaN,8.730943,NaN,NaN,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.042917,NaN,NaN,8.430302,NaN,NaN,NaN,NaN


In [746]:
total_popu_data 

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015.0,2016,2017,2018.0,2019,2020,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.045087,NaN,NaN,6.522714,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.570268,NaN,NaN,3.844880,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.658328,NaN,NaN,10.502686,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.572707,NaN,NaN,9.985572,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.312018,NaN,NaN,5.014911,NaN,NaN,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.508813,NaN,NaN,5.008588,NaN,NaN,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.059653,NaN,NaN,7.614648,NaN,NaN,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.318983,NaN,NaN,5.518178,NaN,NaN,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.213290,NaN,NaN,9.110868,NaN,NaN,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.901536,NaN,NaN,7.777133,NaN,NaN,NaN,NaN


We can now convert both DataFrames into their long representation to allow us to merge both of them with the combined dataset.

In [747]:
bottom_popu_data = pd.melt(bottom_popu_data, id_vars='Geolocation', value_vars=bottom_popu_data.columns [1:]) 

bottom_popu_data.rename(columns = {'value':'10.1.1.1 Bottom 40 percent of the population', 0 : 'Year'}, inplace=True)
bottom_popu_data = bottom_popu_data.astype({'Year':'int'})

total_popu_data = pd.melt(total_popu_data, id_vars='Geolocation', value_vars=total_popu_data.columns [1:]) 

total_popu_data.rename(columns = {'value':'10.1.1.2 Total Population', 0 : 'Year'}, inplace=True)
total_popu_data = total_popu_data.astype({'Year':'int'})

Combining the two different datasets with the currently combined data...

In [748]:
# Adding the 10.1.1.1 dataset with the current combined dataset
combined_data = combined_data.merge(bottom_popu_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)
# Adding the 10.1.1.2 dataset with the current combined dataset
combined_data = combined_data.merge(total_popu_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [749]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",4.1.s1 Completion Rate of elementary and secondary students (Elementary),4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,62.72,70.07,NaN,NaN,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,63.87,68.16,NaN,NaN,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,61.95,70.31,NaN,NaN,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,78.73,73.38,NaN,NaN,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,70.75,72.2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 14.5.1. Coverage of protected areas in relation to marine areas

We will now read the thirteenth dataset.

In [750]:
data = pd.read_csv('data' + '/14.5.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/14.5.1.csv')
data

,14.5.1 Coverage of protected areas in relation to marine areas,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Year,NaN,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019.000000,2020.000000,2021,2022
1,Sub-Indicator,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14.5.1.1 Coverage of protected areas in relati...,PHILIPPINES,..,..,..,..,..,..,..,..,...,..,..,..,1.4121254399999998,..,..,3.143559,3.143559,...,...
3,NaN,..National Capital Region (NCR),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.000108,0.000108,...,...
4,NaN,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.000000,0.000000,...,...
5,NaN,..Region I,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.012083,0.012083,...,...
6,NaN,..Region II,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.280804,0.280804,...,...
7,NaN,..Region III,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.364699,0.364699,...,...
8,NaN,..Region IV-A,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.000610,0.000610,...,...
9,NaN,..MIMAROPA,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.635796,0.635796,...,...


As usual, we drop the irrelevant columns.

In [751]:
data = data.drop (data.index [38:])

We will now edit the data in column 0 & 1 at Index 0 to make the whole Index 0 look like the column headers we want.Then, we set the Index 0 to become the header columns. After this, we drop the Index 0 and the row of NaNs underneath it.

The first column was renamed to `Indicator` as it contains the names of the parts in this dataset.

In [752]:
data.at[0, '14.5.1 Coverage of protected areas in relation to marine areas'] = 'Indicator'
data.at[0, 'Unnamed: 1'] = 'Geolocation'

In [753]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)
data.head()

,Indicator,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019.0,2020.0,2021,2022
0,14.5.1.1 Coverage of protected areas in relati...,PHILIPPINES,..,..,..,..,..,..,..,..,...,..,..,..,1.4121254399999998,..,..,3.143559,3.143559,...,...
1,NaN,..National Capital Region (NCR),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.000108,0.000108,...,...
2,NaN,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.000000,0.000000,...,...
3,NaN,..Region I,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.012083,0.012083,...,...
4,NaN,..Region II,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.280804,0.280804,...,...


To represent the missing values clearly, we change the the '..' or '...' strings to NaN using the np.nan.

In [754]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

As observed in this dataset, we have two parts which are **14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)** and **14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs 1/**. Since we will both need these parts, we will still get both parts to combine with other datasets. However, we will divide them into two different datasets.

For this, we will retain the `Indicator` column first, which contains the name of the parts, for identifying how this dataset will be divided. 

In [755]:
data['Indicator'].unique()
data

,Indicator,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019.0,2020.0,2021,2022
0,14.5.1.1 Coverage of protected areas in relati...,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.4121254399999998,NaN,NaN,3.143559,3.143559,NaN,NaN
1,NaN,..National Capital Region (NCR),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000108,0.000108,NaN,NaN
2,NaN,..Cordillera Administrative Region (CAR),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN
3,NaN,..Region I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.012083,0.012083,NaN,NaN
4,NaN,..Region II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.280804,0.280804,NaN,NaN
5,NaN,..Region III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.364699,0.364699,NaN,NaN
6,NaN,..Region IV-A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000610,0.000610,NaN,NaN
7,NaN,..MIMAROPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.635796,0.635796,NaN,NaN
8,NaN,..Region V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.396713,0.396713,NaN,NaN
9,NaN,..Region VI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.033146,0.033146,NaN,NaN


**14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)** goes to `universe_data` while **14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs 1/** goes to `nipas_data`. 

Since the `nipas_data` will start at Index 18, we will reset it to Index 0 after the division.

In [756]:
universe_data = data [0:18]
nipas_data = data [18:]

In [757]:
nipas_data = nipas_data.reset_index (drop=True)
nipas_data

,Indicator,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019.0,2020.0,2021,2022
0,14.5.1.2 Coverage of protected areas in relati...,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.647,NaN,NaN,1.420000,1.420000,NaN,NaN
1,NaN,..National Capital Region (NCR),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000049,0.000049,NaN,NaN
2,NaN,..Cordillera Administrative Region (CAR),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN
3,NaN,..Region I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.005476,0.005476,NaN,NaN
4,NaN,..Region II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.127265,0.127265,NaN,NaN
5,NaN,..Region III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.165288,0.165288,NaN,NaN
6,NaN,..Region IV-A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000277,0.000277,NaN,NaN
7,NaN,..MIMAROPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.288154,0.288154,NaN,NaN
8,NaN,..Region V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.179797,0.179797,NaN,NaN
9,NaN,..Region VI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.015022,0.015022,NaN,NaN


Since the dividing of the dataset is done, we won't be needing the `Indicator` column anymore. Therefore, we drop them.

In [758]:
universe_data = universe_data.drop('Indicator', axis = 1)
nipas_data = nipas_data.drop('Indicator', axis = 1)

Since the order of the geolocation of each DataFrame is correct, we will initialize it with the region_names for uniformity.

In [759]:
universe_data ['Geolocation'] = region_names
nipas_data ['Geolocation'] = region_names

We can now convert both DataFrames into their long representation to allow us to merge both of them with the combined dataset.

In [760]:
# 14.5.1.1
universe_data = pd.melt(universe_data, id_vars='Geolocation', value_vars=universe_data.columns [1:]) 
universe_data.rename(columns = {'value':'14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)', 0 : 'Year'}, inplace=True)
universe_data = universe_data.astype({'Year':'int'})
# 14.5.1.2
nipas_data = pd.melt(nipas_data, id_vars='Geolocation', value_vars=nipas_data.columns [1:]) 
nipas_data.rename(columns = {'value':'14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs', 0 : 'Year'}, inplace=True)
nipas_data = nipas_data.astype({'Year':'int'})

Combining the two different datasets with the currently combined data...

In [761]:
# Adding the 14.5.1.1 dataset with the current combined dataset
combined_data = combined_data.merge(universe_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)
# Adding the 14.5.1.2 dataset with the current combined dataset
combined_data = combined_data.merge(nipas_data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [762]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",4.1.s1 Completion Rate of elementary and secondary students (Elementary),4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs"
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,62.72,70.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,63.87,68.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,61.95,70.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,78.73,73.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,70.75,72.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 16.1.1 Number of victims of intentional homicide (per 100,000 population)

We will now load the fourteenth dataset.

In [763]:
data = pd.read_csv('data' + '/16.1.1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/16.1.1.csv')
data

,"16.1.1 Number of victims of intentional homicide (per 100,000 population) 1/",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.000000,2017.000000,2018.000000,2019.000000,2020.000000,2021.000000,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,..,..,..,..,..,..,..,..,...,..,..,..,12.110579,8.648427,6.502755,5.660457,4.753062,4.396605,...
3,..National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,..,17.739571,11.632286,5.448315,4.620971,4.766508,3.551425,...
4,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,..,10.348515,3.413519,3.999527,4.799426,3.595654,2.688292,...
5,..Region I,..,..,..,..,..,..,..,..,..,...,..,..,..,11.031909,6.278588,4.441518,3.540128,2.921754,2.785503,...
6,..Region II,..,..,..,..,..,..,..,..,..,...,..,..,..,9.188067,7.405935,5.838454,4.613656,3.198696,2.761486,...
7,..Region III,..,..,..,..,..,..,..,..,..,...,..,..,..,9.704967,6.153771,3.725722,3.502542,2.444428,2.517212,...
8,..Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,..,11.043513,9.088327,6.336361,5.265942,3.991954,4.131750,...
9,..MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,..,7.284387,5.309139,5.203020,4.114172,3.779695,3.362170,...


Now, we will drop the irrelevant rows.

In [764]:
data = data.drop(data.index[20:])

Since Index 0 is almost the same as the column header we want, we will just change the content in the first column to `Geolocation`. This also because the column already containes the regions of the Philippines.

Then, we set the Index 0 to become the header column. After this, we will drop Index 0 and the rows of NANs underneath it since we will not be needing this later.

In [765]:
data.at[0,'16.1.1 Number of victims of intentional homicide (per 100,000 population) 1/'] = 'Geolocation'

In [766]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

We will now check the order of the Geolocation if it is the same as the combined dataset. Then, to make the naming of Geolocation uniformed, we will initialized the Geolocation with region_names.

In [767]:
data ['Geolocation'] = region_names

We will then change the the '..' or '...' strings to NaN.

In [768]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

We can now convert the DataFrame into its long representation using the melt function. This would allow us to merge it with the combined dataset.

In [769]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'16.1.1 Number of victims of intentional homicide (per 100,000 population)', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

In [770]:
data

,Geolocation,Year,"16.1.1 Number of victims of intentional homicide (per 100,000 population)"
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


We will now combine this dataset to the currently combined dataset.

In [771]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [772]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,4.1.s1 Completion Rate of elementary and secondary students (Junior High School),4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)"
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,70.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,68.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,70.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,73.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,72.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 16.1.s1 Number of murder cases

We are now loading our fifteenth dataset.

In [773]:
data = pd.read_csv('data' + '/16.1.s1.csv')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/16.1.s1.csv')
data

,16.1.s1 Number of murder cases,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Year,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022
1,Geolocation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PHILIPPINES,..,..,..,..,..,..,..,..,..,...,..,..,..,12417.0,9009.0,6877.0,6073.0,5170.0,4845.0,...
3,..National Capital Region (NCR),..,..,..,..,..,..,..,..,..,...,..,..,..,2318.0,1543.0,733.0,630.0,658.0,496.0,...
4,..Cordillera Administrative Region (CAR),..,..,..,..,..,..,..,..,..,...,..,..,..,180.0,60.0,71.0,86.0,65.0,49.0,...
5,..Region I,..,..,..,..,..,..,..,..,..,...,..,..,..,560.0,322.0,230.0,185.0,154.0,148.0,...
6,..Region II,..,..,..,..,..,..,..,..,..,...,..,..,..,321.0,262.0,209.0,167.0,117.0,102.0,...
7,..Region III,..,..,..,..,..,..,..,..,..,...,..,..,..,1110.0,718.0,443.0,424.0,301.0,315.0,...
8,..Region IV-A,..,..,..,..,..,..,..,..,..,...,..,..,..,1628.0,1371.0,977.0,829.0,641.0,676.0,...
9,..MIMAROPA,..,..,..,..,..,..,..,..,..,...,..,..,..,219.0,162.0,161.0,129.0,120.0,108.0,...


Dropping the irrelevant rows.

In [774]:
data = data.drop(data.index[20:])

Since Index 0 is almost the same as the column header we want, we will just change the content in the first column to "Geolocation". Then, we set the Index 0 to become the header column. After this, we will drop Index 0 and the rows of NANs underneath it since we will not be needing this later.

In [775]:
data.at[0,'16.1.s1 Number of murder cases'] = 'Geolocation'

In [776]:
data.columns = data.loc[0]
data = data.drop (data.index[0])
data = data.reset_index (drop=True)

data = data.drop (data.index[0])
data = data.reset_index (drop=True)

We will now check the order of the Geolocation if it is the same as the combined dataset. Then, to make the naming of Geolocation uniformed, we will initialized the Geolocation with region_names.

In [777]:
data ['Geolocation'] = region_names

We will then change the the '..' or '...' strings to NaN.

In [778]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

We can now convert the DataFrame into its long representation using the melt function. This would allow us to merge it with the combined dataset.

In [779]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

data.rename(columns = {'value':'16.1.s1 Number of murder cases', 0 : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})

In [780]:
data

,Geolocation,Year,16.1.s1 Number of murder cases
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


We will now combine this dataset to the currently combined dataset.

In [781]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [782]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,4.1.s1 Completion Rate of elementary and secondary students (Senior High School),4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Other Non-SDG datasets
These are datasets that can provide us with more context when exploring the datasets for the Sustainable Development Goals

##### Changes in Inventories, by Region

Loading the sixteenth dataset...

In [783]:
data = pd.read_csv('data' + '/Changes in Inventories, by Region.csv', header=1, delimiter=";")
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/Changes in Inventories, by Region.csv')
data

,Region,At Current Prices 2000,At Current Prices 2001,At Current Prices 2002,At Current Prices 2003,At Current Prices 2004,At Current Prices 2005,At Current Prices 2006,At Current Prices 2007,At Current Prices 2008,...,At Constant 2018 Prices 2011,At Constant 2018 Prices 2012,At Constant 2018 Prices 2013,At Constant 2018 Prices 2014,At Constant 2018 Prices 2015,At Constant 2018 Prices 2016,At Constant 2018 Prices 2017,At Constant 2018 Prices 2018,At Constant 2018 Prices 2019,At Constant 2018 Prices 2020
0,..National Capital Region (NCR),2177317,32407610,47446140,38680890,90746761,-21121052,-62074630,-72647490,-16791691,...,50473166,-49178589,-125589234,-131973801,-120469397,-49930377,-68577782,-8952050,-98460647,-122629716
1,..Cordillera Administrative Region (CAR),-6416286,-3465763,-854248,2816938,5942522,922235,1539829,2224400,1699529,...,9807770,-4145438,-6015136,1353727,-3551322,-6383741,-5976168,-12287003,-17316757,-15694210
2,..Region I (Ilocos Region),-1891391,-415859,-2366347,1563172,11013024,2156933,-12615425,-7528468,-1384595,...,9479368,7907833,18877771,26891050,25584491,20188126,13046690,-344781,3250203,-2671110
3,..Region II (Cagayan Valley),5458610,6710711,7073336,7434120,14501407,21591202,19634112,24258860,18866447,...,-21394026,-17861761,-8124133,1376392,-2498886,-1955958,-3482831,-1033799,8411252,-4722197
4,..Region III (Central Luzon),-52958073,-26135881,-755268,-11413915,-18880906,-4770170,-23168248,-27299916,-1141473,...,23490517,56534528,70926774,70127437,39669136,41643301,49474178,5584131,-18524415,-94205573
5,..Region IV-A (CALABARZON),-111297796,-28830457,-9172498,-5564609,-7379076,30758718,-40148922,-56711613,-51903824,...,-103317518,-140119672,-24785411,8156755,46174756,92008347,134666550,8412077,119066555,-70805144
6,..MIMAROPA Region,2369275,2492120,1885432,1015582,-2997026,1792345,-6738316,-15985915,1808683,...,6152081,-1354071,-2034730,-1780638,2979351,-143590,-14745660,813229,14155277,-1600827
7,..Region V (Bicol Region),5267903,6003300,6176622,133123,2225757,3413493,1929194,350885,5154369,...,-6418017,-5235783,5303467,10733819,1895882,7821486,10128909,-487965,-2329598,-11020680
8,..Region VI (Western Visayas),12238329,12193914,10581660,5708728,6233671,1445825,-6533844,5404782,15823095,...,8356392,-1859172,-15966765,-4018311,-24597200,-35595778,-29316100,-3692731,1997385,-6856575
9,..Region VII (Central Visayas),14163801,23634467,15639464,15346129,14804151,2064083,-22029093,-27281777,-23683350,...,-5194636,-1869955,15528945,11836590,38799214,-14871676,-3752588,-2452424,2274006,-18308635


Since we will be only needng the current prices and we will not be focusing on comparing each record to 2018, we will be dropping the columns with `At Constant 2018 Prices`.

In [784]:
data.drop(data.iloc[:, 22:], inplace=True, axis=1)
data

,Region,At Current Prices 2000,At Current Prices 2001,At Current Prices 2002,At Current Prices 2003,At Current Prices 2004,At Current Prices 2005,At Current Prices 2006,At Current Prices 2007,At Current Prices 2008,...,At Current Prices 2011,At Current Prices 2012,At Current Prices 2013,At Current Prices 2014,At Current Prices 2015,At Current Prices 2016,At Current Prices 2017,At Current Prices 2018,At Current Prices 2019,At Current Prices 2020
0,..National Capital Region (NCR),2177317,32407610,47446140,38680890,90746761,-21121052,-62074630,-72647490,-16791691,...,37960386,-53978335,-25567519,-9498709,-78887754,-61168427,-3527878,-8952050,-170725161,-245064614
1,..Cordillera Administrative Region (CAR),-6416286,-3465763,-854248,2816938,5942522,922235,1539829,2224400,1699529,...,20906816,268953,-2783987,-5031597,-1869509,2845014,-1400306,-12287003,-21374032,-16606356
2,..Region I (Ilocos Region),-1891391,-415859,-2366347,1563172,11013024,2156933,-12615425,-7528468,-1384595,...,22315022,-424963,-891655,2641538,20010380,161771,1990840,-344781,3195818,-2544587
3,..Region II (Cagayan Valley),5458610,6710711,7073336,7434120,14501407,21591202,19634112,24258860,18866447,...,26715349,20349948,-3805473,3196568,-6693576,833107,-1813875,-1033799,4908941,-5842024
4,..Region III (Central Luzon),-52958073,-26135881,-755268,-11413915,-18880906,-4770170,-23168248,-27299916,-1141473,...,9838677,15729819,14493604,-7236367,14886473,8706663,-2730350,5584131,-34811984,-160621025
5,..Region IV-A (CALABARZON),-111297796,-28830457,-9172498,-5564609,-7379076,30758718,-40148922,-56711613,-51903824,...,-65654452,-58510828,-11077119,7465112,-89593345,-7623979,4054462,8412077,98541144,-139794689
6,..MIMAROPA Region,2369275,2492120,1885432,1015582,-2997026,1792345,-6738316,-15985915,1808683,...,5318716,512923,-6946834,580986,-5855580,-10614285,-5470926,813229,14079039,-2663496
7,..Region V (Bicol Region),5267903,6003300,6176622,133123,2225757,3413493,1929194,350885,5154369,...,8723895,7479267,8305364,9615771,-4034708,8943283,2236237,-487965,-1902098,-21815302
8,..Region VI (Western Visayas),12238329,12193914,10581660,5708728,6233671,1445825,-6533844,5404782,15823095,...,21788503,3128975,4547658,-7722981,-15516499,3271575,-2277977,-3692731,3767878,-12138919
9,..Region VII (Central Visayas),14163801,23634467,15639464,15346129,14804151,2064083,-22029093,-27281777,-23683350,...,9876532,-24630912,-15927072,1646648,17167841,3895526,-1260411,-2452424,1950007,-31868725


Also, since the ordering of the Geolocation is different in this dataset, we will be rearranging the rows based on the order of the Geolocation in `region_names`.

In [785]:
data = data.reindex(index=[17,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
data

,Region,At Current Prices 2000,At Current Prices 2001,At Current Prices 2002,At Current Prices 2003,At Current Prices 2004,At Current Prices 2005,At Current Prices 2006,At Current Prices 2007,At Current Prices 2008,...,At Current Prices 2011,At Current Prices 2012,At Current Prices 2013,At Current Prices 2014,At Current Prices 2015,At Current Prices 2016,At Current Prices 2017,At Current Prices 2018,At Current Prices 2019,At Current Prices 2020
17,Philippines,-136845782,24650494,91133746,40200557,115195365,20573012,-178148844,-198481639,-10934733,...,178559066,-42228645,-18218400,6863551,-125424625,-58265758,-15471546,-26943785,-148526300,-699712362
0,..National Capital Region (NCR),2177317,32407610,47446140,38680890,90746761,-21121052,-62074630,-72647490,-16791691,...,37960386,-53978335,-25567519,-9498709,-78887754,-61168427,-3527878,-8952050,-170725161,-245064614
1,..Cordillera Administrative Region (CAR),-6416286,-3465763,-854248,2816938,5942522,922235,1539829,2224400,1699529,...,20906816,268953,-2783987,-5031597,-1869509,2845014,-1400306,-12287003,-21374032,-16606356
2,..Region I (Ilocos Region),-1891391,-415859,-2366347,1563172,11013024,2156933,-12615425,-7528468,-1384595,...,22315022,-424963,-891655,2641538,20010380,161771,1990840,-344781,3195818,-2544587
3,..Region II (Cagayan Valley),5458610,6710711,7073336,7434120,14501407,21591202,19634112,24258860,18866447,...,26715349,20349948,-3805473,3196568,-6693576,833107,-1813875,-1033799,4908941,-5842024
4,..Region III (Central Luzon),-52958073,-26135881,-755268,-11413915,-18880906,-4770170,-23168248,-27299916,-1141473,...,9838677,15729819,14493604,-7236367,14886473,8706663,-2730350,5584131,-34811984,-160621025
5,..Region IV-A (CALABARZON),-111297796,-28830457,-9172498,-5564609,-7379076,30758718,-40148922,-56711613,-51903824,...,-65654452,-58510828,-11077119,7465112,-89593345,-7623979,4054462,8412077,98541144,-139794689
6,..MIMAROPA Region,2369275,2492120,1885432,1015582,-2997026,1792345,-6738316,-15985915,1808683,...,5318716,512923,-6946834,580986,-5855580,-10614285,-5470926,813229,14079039,-2663496
7,..Region V (Bicol Region),5267903,6003300,6176622,133123,2225757,3413493,1929194,350885,5154369,...,8723895,7479267,8305364,9615771,-4034708,8943283,2236237,-487965,-1902098,-21815302
8,..Region VI (Western Visayas),12238329,12193914,10581660,5708728,6233671,1445825,-6533844,5404782,15823095,...,21788503,3128975,4547658,-7722981,-15516499,3271575,-2277977,-3692731,3767878,-12138919


Then, we will proceed to reindexing the rows.

In [786]:
data = data.reset_index (drop=True)
data

,Region,At Current Prices 2000,At Current Prices 2001,At Current Prices 2002,At Current Prices 2003,At Current Prices 2004,At Current Prices 2005,At Current Prices 2006,At Current Prices 2007,At Current Prices 2008,...,At Current Prices 2011,At Current Prices 2012,At Current Prices 2013,At Current Prices 2014,At Current Prices 2015,At Current Prices 2016,At Current Prices 2017,At Current Prices 2018,At Current Prices 2019,At Current Prices 2020
0,Philippines,-136845782,24650494,91133746,40200557,115195365,20573012,-178148844,-198481639,-10934733,...,178559066,-42228645,-18218400,6863551,-125424625,-58265758,-15471546,-26943785,-148526300,-699712362
1,..National Capital Region (NCR),2177317,32407610,47446140,38680890,90746761,-21121052,-62074630,-72647490,-16791691,...,37960386,-53978335,-25567519,-9498709,-78887754,-61168427,-3527878,-8952050,-170725161,-245064614
2,..Cordillera Administrative Region (CAR),-6416286,-3465763,-854248,2816938,5942522,922235,1539829,2224400,1699529,...,20906816,268953,-2783987,-5031597,-1869509,2845014,-1400306,-12287003,-21374032,-16606356
3,..Region I (Ilocos Region),-1891391,-415859,-2366347,1563172,11013024,2156933,-12615425,-7528468,-1384595,...,22315022,-424963,-891655,2641538,20010380,161771,1990840,-344781,3195818,-2544587
4,..Region II (Cagayan Valley),5458610,6710711,7073336,7434120,14501407,21591202,19634112,24258860,18866447,...,26715349,20349948,-3805473,3196568,-6693576,833107,-1813875,-1033799,4908941,-5842024
5,..Region III (Central Luzon),-52958073,-26135881,-755268,-11413915,-18880906,-4770170,-23168248,-27299916,-1141473,...,9838677,15729819,14493604,-7236367,14886473,8706663,-2730350,5584131,-34811984,-160621025
6,..Region IV-A (CALABARZON),-111297796,-28830457,-9172498,-5564609,-7379076,30758718,-40148922,-56711613,-51903824,...,-65654452,-58510828,-11077119,7465112,-89593345,-7623979,4054462,8412077,98541144,-139794689
7,..MIMAROPA Region,2369275,2492120,1885432,1015582,-2997026,1792345,-6738316,-15985915,1808683,...,5318716,512923,-6946834,580986,-5855580,-10614285,-5470926,813229,14079039,-2663496
8,..Region V (Bicol Region),5267903,6003300,6176622,133123,2225757,3413493,1929194,350885,5154369,...,8723895,7479267,8305364,9615771,-4034708,8943283,2236237,-487965,-1902098,-21815302
9,..Region VI (Western Visayas),12238329,12193914,10581660,5708728,6233671,1445825,-6533844,5404782,15823095,...,21788503,3128975,4547658,-7722981,-15516499,3271575,-2277977,-3692731,3767878,-12138919


After this, we will now change the columns names: (1) `Region` to `Geolocation`, (2) `At Current Prices <Year>` to `<Year>`

In [787]:
data.columns = ['Geolocation', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
               '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017','2018', '2019', '2020']

After this, we will insert the region_names in the Geolocation column so that the format of the region_names will fit the ones in the combined data.

In [788]:
data ['Geolocation'] = region_names
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,PHILIPPINES,-136845782,24650494,91133746,40200557,115195365,20573012,-178148844,-198481639,-10934733,...,178559066,-42228645,-18218400,6863551,-125424625,-58265758,-15471546,-26943785,-148526300,-699712362
1,NCR: National Capital Region,2177317,32407610,47446140,38680890,90746761,-21121052,-62074630,-72647490,-16791691,...,37960386,-53978335,-25567519,-9498709,-78887754,-61168427,-3527878,-8952050,-170725161,-245064614
2,CAR: Cordillera Administrative Region,-6416286,-3465763,-854248,2816938,5942522,922235,1539829,2224400,1699529,...,20906816,268953,-2783987,-5031597,-1869509,2845014,-1400306,-12287003,-21374032,-16606356
3,Region 1: Ilocos Region,-1891391,-415859,-2366347,1563172,11013024,2156933,-12615425,-7528468,-1384595,...,22315022,-424963,-891655,2641538,20010380,161771,1990840,-344781,3195818,-2544587
4,Region 2: Cagayan Valley,5458610,6710711,7073336,7434120,14501407,21591202,19634112,24258860,18866447,...,26715349,20349948,-3805473,3196568,-6693576,833107,-1813875,-1033799,4908941,-5842024
5,Region 3: Central Luzon,-52958073,-26135881,-755268,-11413915,-18880906,-4770170,-23168248,-27299916,-1141473,...,9838677,15729819,14493604,-7236367,14886473,8706663,-2730350,5584131,-34811984,-160621025
6,Region 4A: CALABARZON,-111297796,-28830457,-9172498,-5564609,-7379076,30758718,-40148922,-56711613,-51903824,...,-65654452,-58510828,-11077119,7465112,-89593345,-7623979,4054462,8412077,98541144,-139794689
7,MIMAROPA: Southwestern Tagalog Region,2369275,2492120,1885432,1015582,-2997026,1792345,-6738316,-15985915,1808683,...,5318716,512923,-6946834,580986,-5855580,-10614285,-5470926,813229,14079039,-2663496
8,Region 5: Bicol Region,5267903,6003300,6176622,133123,2225757,3413493,1929194,350885,5154369,...,8723895,7479267,8305364,9615771,-4034708,8943283,2236237,-487965,-1902098,-21815302
9,Region 6: Western Visayas,12238329,12193914,10581660,5708728,6233671,1445825,-6533844,5404782,15823095,...,21788503,3128975,4547658,-7722981,-15516499,3271575,-2277977,-3692731,3767878,-12138919


We will then change the the '..' or '...' strings to NaN. This is to represent the missing values.

In [789]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

We can now convert the DataFrame into its long representation using the melt function. This would allow us to merge it with the combined dataset.

In [790]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 
data.rename(columns = {'value':'Changes in Inventories, by Region', 'variable' : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})
data

,Geolocation,Year,"Changes in Inventories, by Region"
0,PHILIPPINES,2000,-136845782
1,NCR: National Capital Region,2000,2177317
2,CAR: Cordillera Administrative Region,2000,-6416286
3,Region 1: Ilocos Region,2000,-1891391
4,Region 2: Cagayan Valley,2000,5458610
...,...,...,...
373,Region 10: Northern Mindanao,2020,-1199474
374,Region 11: Davao Region,2020,-2039156
375,Region 12: SOCCSKSARGEN,2020,-8700862
376,CARAGA: Cordillera Administrative Region,2020,-25602975


Finally, we will now combine this dataset to the currently combined dataset.

In [791]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [792]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,4.c.s2 Number of Technical-Vocational Education and Training (TVET) trainers trained,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases,"Changes in Inventories, by Region"
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-136845782.0
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2177317.0
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6416286.0
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1891391.0
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5458610.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Current Health Expenditure by Region, Growth Rates 

Loading the seventeenth dataset...

In [793]:
data = pd.read_csv('data' + '/Current Health Expenditure by Region, Growth Rates.csv', header=1, delimiter=";")
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/Current Health Expenditure by Region, Growth Rates.csv')
data

,Region,2014-15,2015-16,2016-17,2017-18r,2018-19r,2019-20
0,Total Current Health Expenditure,11.1,10.1,9.6,10.1,10.2,12.6
1,..NCR,3.6,7.9,7.2,11.2,13.7,-15.1
2,..CAR,11.4,9.9,11.0,10.4,14.9,10.4
3,..Ilocos Region,12.4,9.5,10.2,9.8,7.7,26.1
4,..Cagayan Valley,13.9,10.6,5.0,11.7,13.7,23.6
5,..Central Luzon,9.9,10.9,7.6,13.0,11.4,12.3
6,..CALABARZON,1.8,23.6,4.3,46.1,-4.1,-18.8
7,..MIMAROPA,9.9,10.2,22.6,-63.0,163.8,-11.5
8,..Bicol Region,9.7,10.0,5.7,11.8,13.7,16.8
9,..Western Visayas,7.6,11.2,9.1,11.7,3.3,23.9


Since we will only need the data nationwide and per region, we will drop the `Index 0` which contains the Total Current Health Expenditure.

In [794]:
data = data.drop (data.index[0])
data = data.reset_index (drop=True)
data

,Region,2014-15,2015-16,2016-17,2017-18r,2018-19r,2019-20
0,..NCR,3.6,7.9,7.2,11.2,13.7,-15.1
1,..CAR,11.4,9.9,11.0,10.4,14.9,10.4
2,..Ilocos Region,12.4,9.5,10.2,9.8,7.7,26.1
3,..Cagayan Valley,13.9,10.6,5.0,11.7,13.7,23.6
4,..Central Luzon,9.9,10.9,7.6,13.0,11.4,12.3
5,..CALABARZON,1.8,23.6,4.3,46.1,-4.1,-18.8
6,..MIMAROPA,9.9,10.2,22.6,-63.0,163.8,-11.5
7,..Bicol Region,9.7,10.0,5.7,11.8,13.7,16.8
8,..Western Visayas,7.6,11.2,9.1,11.7,3.3,23.9
9,..Central Visayas,6.1,9.7,11.8,13.7,11.7,7.3


Also, since the ordering of the Geolocation is different in this dataset, we will be rearranging the rows based on the order of the Geolocation in region_names. After this, we will reset the index again.

In [795]:
data = data.reindex(index=[17,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
data = data.reset_index (drop=True)
data

,Region,2014-15,2015-16,2016-17,2017-18r,2018-19r,2019-20
0,..Nationwide,27.8,12.2,16.7,4.9,2.5,41.8
1,..NCR,3.6,7.9,7.2,11.2,13.7,-15.1
2,..CAR,11.4,9.9,11.0,10.4,14.9,10.4
3,..Ilocos Region,12.4,9.5,10.2,9.8,7.7,26.1
4,..Cagayan Valley,13.9,10.6,5.0,11.7,13.7,23.6
5,..Central Luzon,9.9,10.9,7.6,13.0,11.4,12.3
6,..CALABARZON,1.8,23.6,4.3,46.1,-4.1,-18.8
7,..MIMAROPA,9.9,10.2,22.6,-63.0,163.8,-11.5
8,..Bicol Region,9.7,10.0,5.7,11.8,13.7,16.8
9,..Western Visayas,7.6,11.2,9.1,11.7,3.3,23.9


After this, we will now change the columns names: (1) `Region` to `Geolocation`, (2) `At Current Prices <Year>` to `<Year>`. 

In [796]:
data.columns = ['Geolocation', '2014', '2015', '2016', '2017','2018', '2019']
data

,Geolocation,2014,2015,2016,2017,2018,2019
0,..Nationwide,27.8,12.2,16.7,4.9,2.5,41.8
1,..NCR,3.6,7.9,7.2,11.2,13.7,-15.1
2,..CAR,11.4,9.9,11.0,10.4,14.9,10.4
3,..Ilocos Region,12.4,9.5,10.2,9.8,7.7,26.1
4,..Cagayan Valley,13.9,10.6,5.0,11.7,13.7,23.6
5,..Central Luzon,9.9,10.9,7.6,13.0,11.4,12.3
6,..CALABARZON,1.8,23.6,4.3,46.1,-4.1,-18.8
7,..MIMAROPA,9.9,10.2,22.6,-63.0,163.8,-11.5
8,..Bicol Region,9.7,10.0,5.7,11.8,13.7,16.8
9,..Western Visayas,7.6,11.2,9.1,11.7,3.3,23.9


As observed, this dataset does not have the records for the years: 2000-2013 and 2020-2022. To allow this dataset to merge with the currently combined dataset easily, we add additional columns for representing the missing years in this dataset.

In [797]:
# For adding columns 2000-2013
col = 1
for i in range(2000,2014):
    data.insert(col, str(i), np.nan, True)
    col+=1
# For adding columns 2020-2022
col = 21
for i in range(2020,2023):
    data.insert(col, str(i), np.nan, True)
    col+=1
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,..Nationwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,27.8,12.2,16.7,4.9,2.5,41.8,NaN,NaN,NaN
1,..NCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.6,7.9,7.2,11.2,13.7,-15.1,NaN,NaN,NaN
2,..CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11.4,9.9,11.0,10.4,14.9,10.4,NaN,NaN,NaN
3,..Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12.4,9.5,10.2,9.8,7.7,26.1,NaN,NaN,NaN
4,..Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.9,10.6,5.0,11.7,13.7,23.6,NaN,NaN,NaN
5,..Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.9,10.9,7.6,13.0,11.4,12.3,NaN,NaN,NaN
6,..CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.8,23.6,4.3,46.1,-4.1,-18.8,NaN,NaN,NaN
7,..MIMAROPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.9,10.2,22.6,-63.0,163.8,-11.5,NaN,NaN,NaN
8,..Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.7,10.0,5.7,11.8,13.7,16.8,NaN,NaN,NaN
9,..Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.6,11.2,9.1,11.7,3.3,23.9,NaN,NaN,NaN


After this, we will insert the region_names in the Geolocation column so that the format of the region_names will fit the ones in the combined data.

In [798]:
data ['Geolocation'] = region_names
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,27.8,12.2,16.7,4.9,2.5,41.8,NaN,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.6,7.9,7.2,11.2,13.7,-15.1,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11.4,9.9,11.0,10.4,14.9,10.4,NaN,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12.4,9.5,10.2,9.8,7.7,26.1,NaN,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.9,10.6,5.0,11.7,13.7,23.6,NaN,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.9,10.9,7.6,13.0,11.4,12.3,NaN,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.8,23.6,4.3,46.1,-4.1,-18.8,NaN,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.9,10.2,22.6,-63.0,163.8,-11.5,NaN,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.7,10.0,5.7,11.8,13.7,16.8,NaN,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.6,11.2,9.1,11.7,3.3,23.9,NaN,NaN,NaN


We will then change the the '..' or '...' strings to NaN. This is to represent the missing values.

In [799]:
for c in data.columns.difference(['Geolocation']):
    data [c].replace(to_replace='..', value= np.nan, inplace= True)
    data [c].replace(to_replace='...', value= np.nan, inplace= True)

We can now convert the DataFrame into its long representation using the melt function. This would allow us to merge it with the combined dataset.

In [800]:
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 
data.rename(columns = {'value':'Current Health Expenditure by Region, Growth Rates', 'variable' : 'Year'}, inplace=True)
data = data.astype({'Year':'int'})
data

,Geolocation,Year,"Current Health Expenditure by Region, Growth Rates"
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


Finally, we will now combine this dataset to the currently combined dataset.

In [801]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [802]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,7.1.1 Proportion of population with access to electricity,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases,"Changes in Inventories, by Region","Current Health Expenditure by Region, Growth Rates"
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-136845782.0,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2177317.0,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6416286.0,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1891391.0,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5458610.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Current Health Expenditure by Region

In [803]:
data = pd.read_csv('data' + '/Current Health Expenditure by Region.csv',header = 1,sep = ';')
# data = pd.read_csv(os.getenv('DSDATA_PROJ') + '/Current Health Expenditure by Region.csv',header = 1,sep = ';')
data

,Region,2014,2015,2016,2017,2018r,2019r,2020
0,Total Current Health Expenditure,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,..NCR,24.7,23.0,22.5,22.3,22.3,23.0,17.3
2,..CAR,2.1,2.1,2.1,2.1,2.1,2.2,2.1
3,..Ilocos Region,4.0,4.0,4.0,4.1,4.0,3.9,4.4
4,..Cagayan Valley,2.3,2.4,2.4,2.3,2.3,2.4,2.6
5,..Central Luzon,10.9,10.7,10.8,10.7,10.9,11.0,11.0
6,..CALABARZON,2.9,2.6,2.9,2.8,3.7,3.2,2.3
7,..MIMAROPA,1.2,1.2,1.2,1.4,0.5,1.1,0.9
8,..Bicol Region,4.2,4.1,4.1,4.0,4.0,4.1,4.3
9,..Western Visayas,6.4,6.2,6.3,6.3,6.4,6.0,6.6


In [804]:
#drop total current health expenditure
data = data.drop (data.index[0])
data

,Region,2014,2015,2016,2017,2018r,2019r,2020
1,..NCR,24.7,23.0,22.5,22.3,22.3,23.0,17.3
2,..CAR,2.1,2.1,2.1,2.1,2.1,2.2,2.1
3,..Ilocos Region,4.0,4.0,4.0,4.1,4.0,3.9,4.4
4,..Cagayan Valley,2.3,2.4,2.4,2.3,2.3,2.4,2.6
5,..Central Luzon,10.9,10.7,10.8,10.7,10.9,11.0,11.0
6,..CALABARZON,2.9,2.6,2.9,2.8,3.7,3.2,2.3
7,..MIMAROPA,1.2,1.2,1.2,1.4,0.5,1.1,0.9
8,..Bicol Region,4.2,4.1,4.1,4.0,4.0,4.1,4.3
9,..Western Visayas,6.4,6.2,6.3,6.3,6.4,6.0,6.6
10,..Central Visayas,5.9,5.6,5.6,5.7,5.9,5.9,5.7


In [805]:
#remove '..' and 'r'
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data.columns = data.columns.str.replace('[r]', '',regex = True)
data

,Region,2014,2015,2016,2017,2018,2019,2020
1,NCR,24.7,23.0,22.5,22.3,22.3,23.0,17.3
2,CAR,2.1,2.1,2.1,2.1,2.1,2.2,2.1
3,Ilocos Region,4.0,4.0,4.0,4.1,4.0,3.9,4.4
4,Cagayan Valley,2.3,2.4,2.4,2.3,2.3,2.4,2.6
5,Central Luzon,10.9,10.7,10.8,10.7,10.9,11.0,11.0
6,CALABARZON,2.9,2.6,2.9,2.8,3.7,3.2,2.3
7,MIMAROPA,1.2,1.2,1.2,1.4,0.5,1.1,0.9
8,Bicol Region,4.2,4.1,4.1,4.0,4.0,4.1,4.3
9,Western Visayas,6.4,6.2,6.3,6.3,6.4,6.0,6.6
10,Central Visayas,5.9,5.6,5.6,5.7,5.9,5.9,5.7


In [806]:
#make nationwide index 0
data = data.iloc[np.arange(-1, len(data)-1)]
data = data.reset_index()
data.drop('index', axis = 1,inplace = True)
data

,Region,2014,2015,2016,2017,2018,2019,2020
0,Nationwide,16.8,19.3,19.7,21.2,20.0,18.6,23.4
1,NCR,24.7,23.0,22.5,22.3,22.3,23.0,17.3
2,CAR,2.1,2.1,2.1,2.1,2.1,2.2,2.1
3,Ilocos Region,4.0,4.0,4.0,4.1,4.0,3.9,4.4
4,Cagayan Valley,2.3,2.4,2.4,2.3,2.3,2.4,2.6
5,Central Luzon,10.9,10.7,10.8,10.7,10.9,11.0,11.0
6,CALABARZON,2.9,2.6,2.9,2.8,3.7,3.2,2.3
7,MIMAROPA,1.2,1.2,1.2,1.4,0.5,1.1,0.9
8,Bicol Region,4.2,4.1,4.1,4.0,4.0,4.1,4.3
9,Western Visayas,6.4,6.2,6.3,6.3,6.4,6.0,6.6


In [807]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
data

,Geolocation,2014,2015,2016,2017,2018,2019,2020
0,PHILIPPINES,16.8,19.3,19.7,21.2,20.0,18.6,23.4
1,NCR: National Capital Region,24.7,23.0,22.5,22.3,22.3,23.0,17.3
2,CAR: Cordillera Administrative Region,2.1,2.1,2.1,2.1,2.1,2.2,2.1
3,Region 1: Ilocos Region,4.0,4.0,4.0,4.1,4.0,3.9,4.4
4,Region 2: Cagayan Valley,2.3,2.4,2.4,2.3,2.3,2.4,2.6
5,Region 3: Central Luzon,10.9,10.7,10.8,10.7,10.9,11.0,11.0
6,Region 4A: CALABARZON,2.9,2.6,2.9,2.8,3.7,3.2,2.3
7,MIMAROPA: Southwestern Tagalog Region,1.2,1.2,1.2,1.4,0.5,1.1,0.9
8,Region 5: Bicol Region,4.2,4.1,4.1,4.0,4.0,4.1,4.3
9,Region 6: Western Visayas,6.4,6.2,6.3,6.3,6.4,6.0,6.6


To follow the format of the combined dataset and to make combining dataset easier, we add columns for years: `2000-2013` and `2020-2022`.

In [808]:
# For adding columns 2000-2013
col = 1
for i in range(2000,2014):
    data.insert(col, str(i), np.nan, True)
    col+=1
# For adding columns 2021 and 2022
data.insert(22, 2021, np.nan, True)
data.insert(23, 2022, np.nan, True)
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,PHILIPPINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,16.8,19.3,19.7,21.2,20.0,18.6,23.4,NaN,NaN
1,NCR: National Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,24.7,23.0,22.5,22.3,22.3,23.0,17.3,NaN,NaN
2,CAR: Cordillera Administrative Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.1,2.1,2.1,2.1,2.1,2.2,2.1,NaN,NaN
3,Region 1: Ilocos Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,4.0,4.0,4.1,4.0,3.9,4.4,NaN,NaN
4,Region 2: Cagayan Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.3,2.4,2.4,2.3,2.3,2.4,2.6,NaN,NaN
5,Region 3: Central Luzon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.9,10.7,10.8,10.7,10.9,11.0,11.0,NaN,NaN
6,Region 4A: CALABARZON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.9,2.6,2.9,2.8,3.7,3.2,2.3,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.2,1.2,1.2,1.4,0.5,1.1,0.9,NaN,NaN
8,Region 5: Bicol Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.2,4.1,4.1,4.0,4.0,4.1,4.3,NaN,NaN
9,Region 6: Western Visayas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.4,6.2,6.3,6.3,6.4,6.0,6.6,NaN,NaN


In [809]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'Current Health Expenditure by Region', 'variable' : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

,Geolocation,Year,Current Health Expenditure by Region
0,PHILIPPINES,2000,NaN
1,NCR: National Capital Region,2000,NaN
2,CAR: Cordillera Administrative Region,2000,NaN
3,Region 1: Ilocos Region,2000,NaN
4,Region 2: Cagayan Valley,2000,NaN
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


Finally, we will now add this dataset with the currently combined dataset.

In [810]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)

In [811]:
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,8.1.1 Annual growth rate of real GDP per capita,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases,"Changes in Inventories, by Region","Current Health Expenditure by Region, Growth Rates",Current Health Expenditure by Region
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-136845782.0,NaN,NaN
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2177317.0,NaN,NaN
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6416286.0,NaN,NaN
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1891391.0,NaN,NaN
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5458610.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Government Final Consumption Expenditure, by Region, Growth Rates
Load next dataset

In [812]:
data = pd.read_csv('data' + '/Government Final Consumption Expenditure, by Region, Growth Rates.csv',header = 1,sep = ';')
data

,Region,At Current Prices 2000-2001,At Current Prices 2001-2002,At Current Prices 2002-2003,At Current Prices 2003-2004,At Current Prices 2004-2005,At Current Prices 2005-2006,At Current Prices 2006-2007,At Current Prices 2007-2008,At Current Prices 2008-2009,...,At Constant 2018 Prices 2010-2011,At Constant 2018 Prices 2011-2012,At Constant 2018 Prices 2012-2013,At Constant 2018 Prices 2013-2014,At Constant 2018 Prices 2014-2015,At Constant 2018 Prices 2015-2016,At Constant 2018 Prices 2016-2017,At Constant 2018 Prices 2017-2018,At Constant 2018 Prices 2018-2019,At Constant 2018 Prices 2019-2020
0,..National Capital Region (NCR),8.3,-1.7,4.7,5.0,5.8,9.1,12.5,9.2,16.6,...,-1.9,14.3,6.0,2.1,5.5,8.3,5.5,13.3,9.8,11.6
1,..Cordillera Administrative Region (CAR),2.3,4.5,0.9,-1.4,14.3,6.1,3.3,3.4,10.0,...,2.7,17.3,0.5,3.6,6.3,6.7,6.7,13.2,6.1,7.2
2,..Region I (Ilocos Region),1.3,6.6,6.8,0.3,11.9,29.2,2.3,0.8,6.4,...,2.1,16.2,2.3,10.7,13.3,12.9,6.9,13.2,6.5,7.6
3,..Region II (Cagayan Valley),4.2,11.5,7.5,4.6,5.7,17.5,11.2,1.8,15.2,...,4.1,5.7,-1.1,5.3,7.8,7.7,6.6,14.3,7.1,8.8
4,..Region III (Central Luzon),2.0,21.1,7.2,19.7,3.8,20.2,21.3,9.1,20.3,...,1.8,17.1,8.1,3.4,8.8,9.4,7.3,13.9,8.8,9.1
5,..Region IV-A (CALABARZON),8.2,15.0,5.9,5.1,19.4,14.1,38.8,0.7,20.3,...,1.2,20.5,2.2,7.9,9.4,10.8,7.5,14.3,9.0,10.1
6,..MIMAROPA Region,3.0,7.8,6.8,-9.0,25.2,18.0,21.4,7.0,16.7,...,-0.7,18.4,-0.3,2.7,4.9,8.9,7.2,13.5,3.4,9.6
7,..Region V (Bicol Region),0.3,3.1,6.7,-5.4,18.4,20.9,20.4,1.3,21.3,...,2.2,20.5,1.7,5.9,8.7,5.5,6.5,9.1,12.9,9.0
8,..Region VI (Western Visayas),7.9,4.2,2.9,-1.9,9.5,18.6,4.1,0.3,20.7,...,2.0,16.2,-0.5,0.7,12.4,12.0,6.8,19.3,7.5,8.1
9,..Region VII (Central Visayas),6.8,11.0,1.8,13.7,0.6,17.7,6.3,7.5,13.2,...,5.0,22.8,12.7,12.1,14.7,13.0,7.8,15.7,8.2,10.6


We remove the '..' at the start of the Region column values then put the last row as the first row to follow the format of region_names

In [813]:
#remove '..' and arrange row
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data = data.iloc[np.arange(-1, len(data)-1)]
data = data.reset_index()
data.drop('index', axis = 1,inplace = True)
#data

We rename the Region column to region_names for consistency then rename the column header Region to Geolocation

In [814]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
#data

We only need the at current price for that year so we drop  the not needed columns. We then format the column names.

In [816]:
data.drop(data.iloc[:, 21:41], inplace = True, axis = 1)
data.columns = data.columns.map(lambda x: x.lstrip('At Current Prices'))
data.columns = data.columns.str[:4]
data.rename(columns = {'Geol': 'Geolocation'},inplace = True)
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,PHILIPPINES,5.7,3.4,4.9,3.9,7.8,13.2,11.9,6.8,16.2,...,7.9,21.1,9.2,7.0,9.1,12.0,10.0,17.3,10.6,12.6
1,NCR: National Capital Region,8.3,-1.7,4.7,5.0,5.8,9.1,12.5,9.2,16.6,...,3.8,19.9,10.3,5.4,6.7,10.9,9.0,17.3,11.3,13.7
2,CAR: Cordillera Administrative Region,2.3,4.5,0.9,-1.4,14.3,6.1,3.3,3.4,10.0,...,8.7,23.0,4.5,6.9,7.5,9.3,10.2,17.1,7.6,9.2
3,Region 1: Ilocos Region,1.3,6.6,6.8,0.3,11.9,29.2,2.3,0.8,6.4,...,8.1,21.9,6.4,14.3,14.5,15.6,10.5,17.1,8.0,9.6
4,Region 2: Cagayan Valley,4.2,11.5,7.5,4.6,5.7,17.5,11.2,1.8,15.2,...,10.2,10.9,2.9,8.7,9.0,10.3,10.1,18.3,8.6,10.9
5,Region 3: Central Luzon,2.0,21.1,7.2,19.7,3.8,20.2,21.3,9.1,20.3,...,7.7,22.8,12.5,6.8,10.0,12.0,10.9,17.9,10.3,11.2
6,Region 4A: CALABARZON,8.2,15.0,5.9,5.1,19.4,14.1,38.8,0.7,20.3,...,7.1,26.4,6.3,11.5,10.5,13.4,11.0,18.3,10.5,12.2
7,MIMAROPA: Southwestern Tagalog Region,3.0,7.8,6.8,-9.0,25.2,18.0,21.4,7.0,16.7,...,5.1,24.2,3.7,6.0,6.0,11.4,10.8,17.4,4.9,11.6
8,Region 5: Bicol Region,0.3,3.1,6.7,-5.4,18.4,20.9,20.4,1.3,21.3,...,8.2,26.4,5.8,9.4,9.9,8.0,10.1,12.9,14.4,11.0
9,Region 6: Western Visayas,7.9,4.2,2.9,-1.9,9.5,18.6,4.1,0.3,20.7,...,8.0,21.9,3.6,4.0,13.6,14.6,10.4,23.4,9.0,10.1


Add missing columns 2020-2022 to be able to merge easily

In [817]:
# For adding columns 2020-2022
col = 21
for i in range(2020,2023):
    data.insert(col, str(i), np.nan, True)
    col+=1

data


,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,PHILIPPINES,5.7,3.4,4.9,3.9,7.8,13.2,11.9,6.8,16.2,...,7.0,9.1,12.0,10.0,17.3,10.6,12.6,NaN,NaN,NaN
1,NCR: National Capital Region,8.3,-1.7,4.7,5.0,5.8,9.1,12.5,9.2,16.6,...,5.4,6.7,10.9,9.0,17.3,11.3,13.7,NaN,NaN,NaN
2,CAR: Cordillera Administrative Region,2.3,4.5,0.9,-1.4,14.3,6.1,3.3,3.4,10.0,...,6.9,7.5,9.3,10.2,17.1,7.6,9.2,NaN,NaN,NaN
3,Region 1: Ilocos Region,1.3,6.6,6.8,0.3,11.9,29.2,2.3,0.8,6.4,...,14.3,14.5,15.6,10.5,17.1,8.0,9.6,NaN,NaN,NaN
4,Region 2: Cagayan Valley,4.2,11.5,7.5,4.6,5.7,17.5,11.2,1.8,15.2,...,8.7,9.0,10.3,10.1,18.3,8.6,10.9,NaN,NaN,NaN
5,Region 3: Central Luzon,2.0,21.1,7.2,19.7,3.8,20.2,21.3,9.1,20.3,...,6.8,10.0,12.0,10.9,17.9,10.3,11.2,NaN,NaN,NaN
6,Region 4A: CALABARZON,8.2,15.0,5.9,5.1,19.4,14.1,38.8,0.7,20.3,...,11.5,10.5,13.4,11.0,18.3,10.5,12.2,NaN,NaN,NaN
7,MIMAROPA: Southwestern Tagalog Region,3.0,7.8,6.8,-9.0,25.2,18.0,21.4,7.0,16.7,...,6.0,6.0,11.4,10.8,17.4,4.9,11.6,NaN,NaN,NaN
8,Region 5: Bicol Region,0.3,3.1,6.7,-5.4,18.4,20.9,20.4,1.3,21.3,...,9.4,9.9,8.0,10.1,12.9,14.4,11.0,NaN,NaN,NaN
9,Region 6: Western Visayas,7.9,4.2,2.9,-1.9,9.5,18.6,4.1,0.3,20.7,...,4.0,13.6,14.6,10.4,23.4,9.0,10.1,NaN,NaN,NaN


Then, we can now convert our DataFrame into its long representation using the melt function. As in the processing of the previous datasets, we would have to rename the column names as they are not descriptive enough.

In [818]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'Consumption Expenditure GR', 'variable' : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

,Geolocation,Year,Consumption Expenditure GR
0,PHILIPPINES,2000,5.7
1,NCR: National Capital Region,2000,8.3
2,CAR: Cordillera Administrative Region,2000,2.3
3,Region 1: Ilocos Region,2000,1.3
4,Region 2: Cagayan Valley,2000,4.2
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


We use the merge function to use outer join to merge the two datasets.

In [819]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,10.1.1.1 Bottom 40 percent of the population,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases,"Changes in Inventories, by Region","Current Health Expenditure by Region, Growth Rates",Current Health Expenditure by Region,Consumption Expenditure GR
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-136845782.0,NaN,NaN,5.7
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2177317.0,NaN,NaN,8.3
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-6416286.0,NaN,NaN,2.3
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1891391.0,NaN,NaN,1.3
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5458610.0,NaN,NaN,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Government Final Consumption Expenditure, by Region, Percent Share
Load the next dataset

In [820]:
data = pd.read_csv('data' + '/Government Final Consumption Expenditure, by Region, Percent Share.csv',header = 1,sep = ';')
data

,Region,At Current Prices 2000,At Current Prices 2001,At Current Prices 2002,At Current Prices 2003,At Current Prices 2004,At Current Prices 2005,At Current Prices 2006,At Current Prices 2007,At Current Prices 2008,...,At Constant 2018 Prices 2011,At Constant 2018 Prices 2012,At Constant 2018 Prices 2013,At Constant 2018 Prices 2014,At Constant 2018 Prices 2015,At Constant 2018 Prices 2016,At Constant 2018 Prices 2017,At Constant 2018 Prices 2018,At Constant 2018 Prices 2019,At Constant 2018 Prices 2020
0,..National Capital Region (NCR),45.9,47.0,44.8,44.7,45.1,44.3,42.7,42.9,43.9,...,42.1,41.7,42.1,41.5,40.6,40.2,39.8,39.8,40.0,40.4
1,..Cordillera Administrative Region (CAR),2.7,2.7,2.7,2.6,2.5,2.6,2.4,2.3,2.2,...,2.1,2.1,2.0,2.0,2.0,1.9,1.9,1.9,1.9,1.8
2,..Region I (Ilocos Region),4.0,3.8,3.9,4.0,3.9,4.0,4.6,4.2,4.0,...,3.7,3.7,3.6,3.9,4.0,4.2,4.2,4.2,4.1,4.0
3,..Region II (Cagayan Valley),2.4,2.4,2.6,2.6,2.7,2.6,2.7,2.7,2.6,...,2.5,2.3,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.1
4,..Region III (Central Luzon),4.3,4.1,4.8,4.9,5.7,5.5,5.8,6.3,6.4,...,6.7,6.8,7.0,7.0,7.0,7.0,7.1,7.1,7.1,7.0
5,..Region IV-A (CALABARZON),4.4,4.5,5.0,5.0,5.1,5.6,5.7,7.0,6.6,...,6.9,7.2,7.0,7.3,7.4,7.5,7.5,7.6,7.6,7.6
6,..MIMAROPA Region,2.0,2.0,2.1,2.1,1.8,2.1,2.2,2.4,2.4,...,2.3,2.4,2.3,2.2,2.2,2.2,2.2,2.2,2.1,2.1
7,..Region V (Bicol Region),3.6,3.5,3.5,3.5,3.2,3.5,3.8,4.0,3.8,...,4.0,4.2,4.1,4.1,4.2,4.0,4.0,3.9,4.0,3.9
8,..Region VI (Western Visayas),5.0,5.1,5.1,5.0,4.8,4.8,5.1,4.7,4.4,...,4.6,4.7,4.4,4.3,4.5,4.6,4.6,4.8,4.8,4.7
9,..Region VII (Central Visayas),2.7,2.8,3.0,2.9,3.2,3.0,3.1,2.9,2.9,...,3.0,3.2,3.5,3.8,4.0,4.1,4.2,4.3,4.2,4.2


We remove the '..' at the start of the Region column values then put the last row as the first row to follow the format of region_names

In [821]:
#remove '..' and arrange row
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data = data.iloc[np.arange(-1, len(data)-1)]
data = data.reset_index()
data.drop('index', axis = 1,inplace = True)
#data

We rename the Region column to region_names for consistency then rename the column header Region to Geolocation

In [822]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
# data

We only need the at current price for that year so we drop  the not needed columns. We then format the column names.

In [823]:
data.drop(data.iloc[:, 22:43], inplace = True, axis = 1)
data.columns = data.columns.map(lambda x: x.lstrip('At Current Prices'))
data.columns = data.columns.str[:4]
data.rename(columns = {'Geol': 'Geolocation'},inplace = True)
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,PHILIPPINES,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,NCR: National Capital Region,45.9,47.0,44.8,44.7,45.1,44.3,42.7,42.9,43.9,...,42.1,41.7,42.1,41.5,40.6,40.2,39.8,39.8,40.0,40.4
2,CAR: Cordillera Administrative Region,2.7,2.7,2.7,2.6,2.5,2.6,2.4,2.3,2.2,...,2.1,2.1,2.0,2.0,2.0,1.9,1.9,1.9,1.9,1.8
3,Region 1: Ilocos Region,4.0,3.8,3.9,4.0,3.9,4.0,4.6,4.2,4.0,...,3.7,3.7,3.6,3.9,4.0,4.2,4.2,4.2,4.1,4.0
4,Region 2: Cagayan Valley,2.4,2.4,2.6,2.6,2.7,2.6,2.7,2.7,2.6,...,2.5,2.3,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.1
5,Region 3: Central Luzon,4.3,4.1,4.8,4.9,5.7,5.5,5.8,6.3,6.4,...,6.7,6.8,7.0,7.0,7.0,7.0,7.1,7.1,7.1,7.0
6,Region 4A: CALABARZON,4.4,4.5,5.0,5.0,5.1,5.6,5.7,7.0,6.6,...,6.9,7.2,7.0,7.3,7.4,7.5,7.5,7.6,7.6,7.6
7,MIMAROPA: Southwestern Tagalog Region,2.0,2.0,2.1,2.1,1.8,2.1,2.2,2.4,2.4,...,2.3,2.4,2.3,2.2,2.2,2.2,2.2,2.2,2.1,2.1
8,Region 5: Bicol Region,3.6,3.5,3.5,3.5,3.2,3.5,3.8,4.0,3.8,...,4.0,4.2,4.1,4.1,4.2,4.0,4.0,3.9,4.0,3.9
9,Region 6: Western Visayas,5.0,5.1,5.1,5.0,4.8,4.8,5.1,4.7,4.4,...,4.6,4.7,4.4,4.3,4.5,4.6,4.6,4.8,4.8,4.7


Add missing columns 2021-2022 to be able to merge easily

In [824]:
# For adding columns 2021-2022
col = 22
for i in range(2021,2023):
    data.insert(col, str(i), np.nan, True)
    col+=1

Then, we can now convert our DataFrame into its long representation using the melt function. As in the processing of the previous datasets, we would have to rename the column names as they are not descriptive enough.

In [825]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'Consumption Expenditure %', 'variable' : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

,Geolocation,Year,Consumption Expenditure %
0,PHILIPPINES,2000,100.0
1,NCR: National Capital Region,2000,45.9
2,CAR: Cordillera Administrative Region,2000,2.7
3,Region 1: Ilocos Region,2000,4.0
4,Region 2: Cagayan Valley,2000,2.4
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


We use the merge function to use outer join to merge the two datasets.

In [826]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,10.1.1.2 Total Population,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases,"Changes in Inventories, by Region","Current Health Expenditure by Region, Growth Rates",Current Health Expenditure by Region,Consumption Expenditure GR,Consumption Expenditure %
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-136845782.0,NaN,NaN,5.7,100.0
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2177317.0,NaN,NaN,8.3,45.9
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-6416286.0,NaN,NaN,2.3,2.7
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1891391.0,NaN,NaN,1.3,4.0
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5458610.0,NaN,NaN,4.2,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Gross Capital Formation, by Region
Load the next dataset

In [827]:
data = pd.read_csv('data' + '/Gross Capital Formation, by Region.csv',header = 1,sep = ';')
data

,Region,At Current Prices 2000,At Current Prices 2001,At Current Prices 2002,At Current Prices 2003,At Current Prices 2004,At Current Prices 2005,At Current Prices 2006,At Current Prices 2007,At Current Prices 2008,...,At Constant 2018 Prices 2011,At Constant 2018 Prices 2012,At Constant 2018 Prices 2013,At Constant 2018 Prices 2014,At Constant 2018 Prices 2015,At Constant 2018 Prices 2016,At Constant 2018 Prices 2017,At Constant 2018 Prices 2018,At Constant 2018 Prices 2019,At Constant 2018 Prices 2020
0,..National Capital Region (NCR),203930819,288023206,312792420,350366278,445396815,364638998,350945836,410794316,541772989,...,795953279,814136171,899258808,943457629,1055777100,1334869204,1364654442,1640251453,1629320588,1102162331
1,..Cordillera Administrative Region (CAR),13865180,13448285,16806013,19788755,24257807,20432006,25385665,26901466,29704835,...,43410767,29012559,33620165,35098035,30308209,24558165,33059389,32266092,26362454,23041977
2,..Region I (Ilocos Region),24454284,26821755,29627138,32967294,45222018,39338515,32127165,41310895,50763801,...,82663294,85433538,104875107,120705416,128404394,160871351,181860608,193759545,220744366,171771884
3,..Region II (Cagayan Valley),32773347,34873457,37349804,40755790,51068600,58370199,68595346,71904291,73649021,...,42835117,52381575,67970385,83988890,92410049,102800111,120125423,134311405,147930411,100972112
4,..Region III (Central Luzon),8037710,35506256,63533211,64973784,68231368,92419886,84806309,92665051,138098487,...,254138225,309828802,353024974,371092410,406615615,473831044,585347897,590017254,610620576,342676989
5,..Region IV-A (CALABARZON),18214696,76941117,108934789,111408235,117422285,173261985,120393803,119653336,149756689,...,175971083,160028504,288993216,323836715,451566485,630840253,767416654,712401187,770805671,377997089
6,..MIMAROPA Region,10234683,11200127,14899234,12029760,9464169,15527012,10027999,3017260,22411802,...,34062851,28545616,32125182,47669461,50527960,49834380,49231467,78995250,86362820,55876531
7,..Region V (Bicol Region),23129693,23889005,26003348,23452407,28387601,30110212,31513023,37647782,47880014,...,51953786,65112644,89509800,108718089,125633690,142075341,160202016,181534196,198313314,140428210
8,..Region VI (Western Visayas),46342201,44831024,46626457,45601881,48709503,44867130,52570827,61931227,73382845,...,90978389,97801511,84268564,86501669,91657960,90785992,111771723,155804331,171809008,123668823
9,..Region VII (Central Visayas),65471807,74574164,72065505,70933637,76422804,64122060,52741677,51229324,66644077,...,122005475,157231413,174373898,183162167,195165138,211462656,230970904,257710115,299306342,184535636


We remove the '..' at the start of the Region column values then put the last row as the first row to follow the format of region_names

In [828]:
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data = data.iloc[np.arange(-1, len(data)-1)]
data = data.reset_index()
data.drop('index', axis = 1,inplace = True)
#data

We rename the Region column to region_names for consistency then rename the column header Region to Geolocation

In [829]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
#data

We only need the at current price for that year so we drop  the not needed columns. We then format the column names.

In [830]:
data.drop(data.iloc[:, 22:43], inplace = True, axis = 1)
data.columns = data.columns.map(lambda x: x.lstrip('At Current Prices'))
data.columns = data.columns.str[:4]
data.rename(columns = {'Geol': 'Geolocation'},inplace = True)
data

,Geolocation,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,PHILIPPINES,579938180,762429457,890086990,921328434,1103698971,1098633998,1049071426,1160979516,1526893379,...,2103996324,2163531693,2487510204,2763392839,2975815372,3725365802,4231677010,4959105466,5153068851,3118277451
1,NCR: National Capital Region,203930819,288023206,312792420,350366278,445396815,364638998,350945836,410794316,541772989,...,720272869,758973608,957586594,1055722012,1072923782,1268840988,1391592148,1640251453,1598461505,974094418
2,CAR: Cordillera Administrative Region,13865180,13448285,16806013,19788755,24257807,20432006,25385665,26901466,29704835,...,49944680,29548738,33081560,24485878,29049288,32105379,35881923,32266092,23666821,23904356
3,Region 1: Ilocos Region,24454284,26821755,29627138,32967294,45222018,39338515,32127165,41310895,50763801,...,84747579,68070088,75736595,88877912,113489548,131580148,163128777,193759545,228616710,183511880
4,Region 2: Cagayan Valley,32773347,34873457,37349804,40755790,51068600,58370199,68595346,71904291,73649021,...,80940288,79946525,62731146,75028145,77480450,95196604,113995195,134311405,146410694,103694574
5,Region 3: Central Luzon,8037710,35506256,63533211,64973784,68231368,92419886,84806309,92665051,138098487,...,216166904,244530920,274814176,273085629,351251636,421665953,513606387,590017254,610978504,285483239
6,Region 4A: CALABARZON,18214696,76941117,108934789,111408235,117422285,173261985,120393803,119653336,149756689,...,217059209,231742975,290367514,320280912,303867486,517220942,630500072,712401187,766853055,311327450
7,MIMAROPA: Southwestern Tagalog Region,10234683,11200127,14899234,12029760,9464169,15527012,10027999,3017260,22411802,...,28886357,24934114,21427705,41529678,36665567,36490353,55532730,78995250,88406153,57697156
8,Region 5: Bicol Region,23129693,23889005,26003348,23452407,28387601,30110212,31513023,37647782,47880014,...,59544473,69740827,83011228,98493385,104978578,134334431,142545107,181534196,202053823,136305863
9,Region 6: Western Visayas,46342201,44831024,46626457,45601881,48709503,44867130,52570827,61931227,73382845,...,93884238,95477787,101336678,77105459,95430431,119446427,132134843,155804331,176527730,116283240


Add missing columns 2021-2022 to be able to merge easily

In [831]:
# For adding columns 2021-2022
col = 22
for i in range(2021,2023):
    data.insert(col, str(i), np.nan, True)
    col+=1

Then, we can now convert our DataFrame into its long representation using the melt function. As in the processing of the previous datasets, we would have to rename the column names as they are not descriptive enough.

In [832]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'Gross Capital Formation', 'variable' : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

,Geolocation,Year,Gross Capital Formation
0,PHILIPPINES,2000,579938180.0
1,NCR: National Capital Region,2000,203930819.0
2,CAR: Cordillera Administrative Region,2000,13865180.0
3,Region 1: Ilocos Region,2000,24454284.0
4,Region 2: Cagayan Valley,2000,32773347.0
...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN
410,Region 11: Davao Region,2022,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN


We use the merge function to use outer join to merge the two datasets.

In [833]:
combined_data = combined_data.merge(data, how = 'outer', on = ['Geolocation', 'Year'])
combined_data = combined_data.reset_index (drop=True)
combined_data

,Geolocation,Year,1.2.1. Proportion of population living below the national poverty line,1.4.1p5 Net Enrolment Rate in elementary,1.4.1p6 Net Enrolment Rate in secondary education (Junior High School),1.4.1p6 Net Enrolment Rate in secondary education (Senior High School),1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies,"3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)",3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods,"3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group",...,"14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)","14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs","16.1.1 Number of victims of intentional homicide (per 100,000 population)",16.1.s1 Number of murder cases,"Changes in Inventories, by Region","Current Health Expenditure by Region, Growth Rates",Current Health Expenditure by Region,Consumption Expenditure GR,Consumption Expenditure %,Gross Capital Formation
0,PHILIPPINES,2000,NaN,96.77,66.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-136845782.0,NaN,NaN,5.7,100.0,579938180.0
1,NCR: National Capital Region,2000,NaN,101,79.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2177317.0,NaN,NaN,8.3,45.9,203930819.0
2,CAR: Cordillera Administrative Region,2000,NaN,94.42,71.19,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-6416286.0,NaN,NaN,2.3,2.7,13865180.0
3,Region 1: Ilocos Region,2000,NaN,97.73,87.51,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1891391.0,NaN,NaN,1.3,4.0,24454284.0
4,Region 2: Cagayan Valley,2000,NaN,95.65,77.11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5458610.0,NaN,NaN,4.2,2.4,32773347.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Region 10: Northern Mindanao,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Region 11: Davao Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,Region 12: SOCCSKSARGEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,CARAGA: Cordillera Administrative Region,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Gross Regional Domestic Product, by Region

In [ ]:
data = pd.read_csv('data' + '/Gross Regional Domestic Product, by Region.csv',header = 1,sep = ';')
data

In [ ]:
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data = data.iloc[np.arange(-1, len(data)-1)]
data = data.reset_index()
data.drop('index', axis = 1,inplace = True)
data

In [ ]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
data

##### Population, by Region

In [ ]:
data = pd.read_csv('data' + '/Population, by Region.csv',header = 1,sep = ';')
data

In [ ]:
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data = data.iloc[np.arange(-1, len(data)-1)]
data = data.reset_index()
data.drop('index', axis = 1,inplace = True)
data

In [ ]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
data

In [ ]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'Population', 'variable' : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

##### Primary Drop-out rates by Region, Sex and Year

In [ ]:
data = pd.read_csv('data' + '/Primary Drop-out rates by Region, Sex and Year.csv',header = 1,sep = ';')
data

In [ ]:
# renames the data in the Geolocation for consistency
data['Region'] = region_names
data.set_index('Region')
data = data.reset_index(drop=True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
data

In [ ]:
data.drop(data.iloc[:, 11:31], inplace = True, axis = 1)
  
data

In [ ]:
data.columns = data.columns.map(lambda x: x.lstrip('Both Sexes '))
data

In [ ]:
# converting from a wide representation to a long representation
data = pd.melt(data, id_vars='Geolocation', value_vars=data.columns [1:]) 

# renaming the columns into a more readable anmes
data.rename(columns = {'value':'Drop-out rate', 'variable' : 'Year'}, inplace=True)

# making the year type into integer
data = data.astype({'Year':'int'})

data

##### Quarterly Producer Price Index for Agriculture (First Quarter 2018 to Third Quarter 2021)

In [ ]:
data = pd.read_csv('data' + '/Quarterly Producer Price Index for Agriculture (2018=100) _ First Quarter 2018 to Third Quarter 2021.csv',header = 1,sep = ';')
data[data['Commodity'] == 'AGRICULTURE']

In [ ]:
data['Region'] = data['Region'].map(lambda x: x.lstrip('..'))
data['Commodity'] = data['Commodity'].map(lambda x: x.lstrip('..'))
data['Commodity'] = data['Commodity'].map(lambda x: x.lstrip('….'))
data

In [ ]:
data.drop(columns =['2021 Fourth Quarter (Oct-Dec)', '2021 Average (Jan-Dec)'],inplace = True)
data.rename(columns = {'Region': 'Geolocation'},inplace = True)
data

## Data Cleaning
There are four steps for the cleaning of the combined dataset: (1) the dropping of the rows wherein all the values of the indicator columns are **NaN**s, (2) the fixing of the data types of the columns, (3) the dropping of duplicated rows, and (4) the cleaning of the individual columns.

### Dropping of rows that has all **NaN** values
The first thing that we would do is to drop the rows that only have **NaN** values. This means that for that specific region in that specific year, there is no data that is collected for any of the indicators, thus, we would not be able to derive any knowledge from it.

Using the combination of the isna and sum functions, we would be able to see the total number of **NaN** values a specific row has.

In [ ]:
combined_data.isna().sum(axis = 1).sort_values(ascending=False)

From the result above, we can see that there are rows that have all **NaN** values (i.e., where the number of **NaN** values outputted is equal to the number of the columns for indicators). Since we know that the `Geolocation` and `Year` column does not have any **NaN** values, we would set a threshold of 3 (which means that if there are at least three non-NaN values, the row would not be dropped ) in the [`dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) function.

In [ ]:
combined_data = combined_data.dropna(axis = 0, thresh = 3)

With this, we would have a new DataFrame that has 377 rows, with the `Year` having a range of from 2000 to 2021.

In [ ]:
combined_data['Year'].describe()

In [ ]:
combined_data

### Fixing the Data Types of the Columns
Using the [`dtypes`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html) property, we would see that some indicators columns are **object**-types. As we know that all columns except for the `Geolocation` and `Year` are supposed to be **float64** columns, we would need to convert these objects.

In [ ]:
combined_data.dtypes

For each of the column that are not the `Geolocation` and `Year` columns, their datatypes are checker. In the scenario that they are not **float64**, the function [`astype`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html) was used in order to convert it to float. Even though we are sure that all of the values in these columns can be transformed to float as this was its original value in the csv file, the parameter `errors` was still set to **raise** for validation.

In [ ]:
for x in combined_data.columns.difference(['Geolocation', 'Year']):
    if combined_data[x].dtypes != 'float64':
        combined_data.loc[:, x] = combined_data[x].astype(float, errors = 'raise')

Using the [`info`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) function, we would see that all the indicator columns are now **float64**.

In [ ]:
combined_data.info()

### Dropping of Duplicated Rows
Using a combination of [`duplicated`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) and [`sum`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html), we would be able to see how many rows are duplicated and should be dropped.

In [ ]:
combined_data.duplicated().sum()

As the combination of these functions outputted the number 0, then we can conclude that each of the rows are unique. This means that we would not have to drop any of the rows.

### Cleaning of Each Columns
As each of the column came from different datasets, we would be checking and cleaning the values for each of the column.

#### 1.2.1. Proportion of population living below the national poverty line
For this column, we would be using the [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function in order to check if we have an outliner. This is due to the fact that we are expecting a value of 0 to 10, as we are talking about proportion or percentage.

In [ ]:
combined_data['1.2.1. Proportion of population living below the national poverty line'].describe()

From what we can see, the minimum and maximum values of the columns are within the range of values that we expected from this column. Thus, there are no outliers that we need to remove or drop.

#### 1.4.1p5 Net Enrolment Rate in elementary
Just like in the first column, we would be using the [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function in order to check the value range of the variable. 

According to the Philippine Statistics Authority (n.d.), the formula for net enrollment rate in elementary is defined as total enrollment of aged six to 11, divided by the population of kids of the same age, and then multiplied by 100. For this, we are expecting a value of 0 to 100, as we are talking about a percentage of a population: we cannot have more children enrolled than the total population of kids. 

In [ ]:
combined_data['1.4.1p5 Net Enrolment Rate in elementary'].describe()

As we can see, the maximum value of this column is higher than 100, which can be concerning as the unit of measurement set by United Nations for all of the countries in this indicators is percentage. Thus, these might be error in encodings.

Let us check all of the rows which has values higher than 100 for this indicator.

In [ ]:
combined_data[combined_data['1.4.1p5 Net Enrolment Rate in elementary'] > 100]

As we can see, there are 18 rows which has more than 100% value for the `1.4.1p5 Net Enrolment Rate in elementary`. In order to prevent these values from skewing the data in the scenario that it is used for computation, these values are instead turned into **NaN**s.

In [ ]:
combined_data.loc[combined_data['1.4.1p5 Net Enrolment Rate in elementary'] > 100, '1.4.1p5 Net Enrolment Rate in elementary'] = np.nan

Now, we can see that all of the values for this column are now within the range that we expected.

In [ ]:
combined_data[combined_data['1.4.1p5 Net Enrolment Rate in elementary'] > 100]

#### 1.4.1p6 Net Enrolment Rate in secondary education (Junior High School)

As we have the same expectations in the second dataset, the [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function would be used in order to check if there are outliers or values that are outside of the range.

In [ ]:
combined_data['1.4.1p6 Net Enrolment Rate in secondary education (Junior High School)'].describe()

From the minimum and maximum value, we can see that the range of values are within the expected values.

#### 1.4.1p6 Net Enrolment Rate in secondary education (Senior High School)
Next, in this column, we would be using the [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function for the same purpose: checking if the maximum and minimum values are within the range we expected.

In [ ]:
combined_data['1.4.1p6 Net Enrolment Rate in secondary education (Senior High School)'].describe()

Based on the output, we can see that the minimum and maximum are within the range.

However, another expectation that we have from this column is that the rows that are not **NaN** have a value of **2016 - onwards** for the `Year` column. This is due to the fact that the Senior High School years was only added from 2016. Thus, if there are values for years lower than this, we would need to turn it to **NaN**.

To check this, we can use a mixture of the [`isnull`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html) function and the [`unique`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html) function. Using the negation of the [`isnull`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html) function, we can only return rows that are not missing. Then, using the [`unique`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html) function, we can return the unique values of the `Year` column of the previously returned rows.

In [ ]:
combined_data[~combined_data['1.4.1p6 Net Enrolment Rate in secondary education (Senior High School)'].isnull()]['Year'].unique()

We can see that the values of the `Year` column of the rows that are not **NaN** for this column are what we expected.

#### 1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies

As this column talks about proportion, we are expecting a value from 0 to 100 again. This means that we can check it using the same function as the previous columns (i.e., the [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function), in order to recheck this using the returned minimum and maximum values.

In [ ]:
combined_data['1.5.4 Proportion of local governments that adopt and implement local disaster risk reduction strategies in line with national disaster risk reduction strategies'].describe()

Since the maximum is 100 and the minimum is not less than 0, then we can conclude that there are no values that are outside of the accepted range.

#### 3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)

In [ ]:
combined_data['3.4.1 Mortality rate attributed to cardiovascular disease, cancer, diabetes or chronic respiratory disease (Total data)'].describe()

#### 3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods

In [ ]:
combined_data['3.7.1 Proportion of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied [provided] with modern methods'].describe()

#### 3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group

In [ ]:
combined_data['3.7.2 Adolescent birth rate aged 15-19 years per 1,000 women in that age group'].describe()

#### 4.1.s1 Completion Rate of elementary and secondary students (Elementary)

In [ ]:
combined_data['4.1.s1 Completion Rate of elementary and secondary students (Elementary)'].describe()

#### 4.1.s1 Completion Rate of elementary and secondary students (Junior High School)

In [ ]:
combined_data['4.1.s1 Completion Rate of elementary and secondary students (Junior High School)'].describe()

#### 4.1.s1 Completion Rate of elementary and secondary students (Senior High School)

In [ ]:
combined_data['4.1.s1 Completion Rate of elementary and secondary students (Senior High School)'].describe()

In [ ]:
combined_data[~combined_data['4.1.s1 Completion Rate of elementary and secondary students (Senior High School)'].isnull()]['Year'].unique()

#### 7.1.1 Proportion of population with access to electricity

In [ ]:
combined_data['7.1.1 Proportion of population with access to electricity'].describe()

In [ ]:
combined_data[combined_data['7.1.1 Proportion of population with access to electricity'] > 100]

In [ ]:
combined_data.loc[combined_data['7.1.1 Proportion of population with access to electricity'] > 100, '7.1.1 Proportion of population with access to electricity'] = np.nan

In [ ]:
combined_data[combined_data['7.1.1 Proportion of population with access to electricity'] > 100]

#### 10.1.1.1 Growth rates of household expenditure or income per capita among the bottom 40 per cent of the population

#### 10.1.1.2 Growth rates of household expenditure or income per capita among the Total Population

#### 14.5.1.1 Coverage of protected areas in relation to marine areas, Universe (in million hectares)

#### 14.5.1.2 Coverage of protected areas in relation to marine areas, NIPAS ans Locally managed MPAs

#### 16.1.1 Number of victims of intentional homicide (per 100,000 population)

#### 16.1.s1 Number of murder cases', 'Changes in Inventories, by Region

#### Current Health Expenditure by Region, Growth Rates

#### Current Health Expenditure by Region

## Exploratory Data Analysis
With the combined dataset, there is a substantial amount of raw data to process and analyze. Before performing any statistical analysis, it is good practice to do exploratory data analysis to observe patterns and detect any outliers in the dataset. With this, we can properly identify particular relationships between specific variables.

### Per year, what region has the lowest proportion value of the population living below the national poverty line?
To answer this questio, we would be utilizing three columns from the combined DataFrame: (1) `Geolocation`, (2) `Year`. (3) `1.2.1. Proportion of population living below the national poverty line`. However, since we aim to get the lowest proportion value per year, we would first need to group the rows, using the [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function. 

Once the rows has been grouped, per group, we would be getting the [`min`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html)imum value for the `1.2.1. Proportion of population living below the national poverty line` column. Then, as we know from the cleaning that there are years without values for this column, we would only be getting the years that has not **NaN** as its minimum value, using the [`notna`](https://pandas.pydata.org/docs/reference/api/pandas.Series.notna.html) function.

In [ ]:
grouped_by_year = combined_data.groupby(['Year'])['1.2.1. Proportion of population living below the national poverty line'].min()
grouped_by_year = grouped_by_year[grouped_by_year.notna()]
grouped_by_year

Then, once we got the minimum values for this column, we can use this to get the rows that has this minimum value.

In [ ]:
combined_data[combined_data ['1.2.1. Proportion of population living below the national poverty line'].isin(grouped_by_year.values)]

From the DataFrame, we can see that the **National Capital Region** has the lowest proportion value of the population living below the national poverty line for both of the years, and that it even decreased in the year 2018. 

Let us cross-check this using bar graph, wherein we would be able to see the proportion value of the population living below the national poverty line per region clearly. 

To do this, let us first get all the rows for **2015** and **2018**.

In [ ]:
data_2015 = combined_data[combined_data['Year'] == 2015]
data_2018 = combined_data[combined_data['Year'] == 2018]

Let us plot the data from 2015 into a bar graph using the [`plot`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) function.

In [ ]:
ax1 = data_2015['1.2.1. Proportion of population living below the national poverty line'].plot(figsize=(8, 6), kind='bar', width=0.5)

ax1.set_xticklabels(data_2015['Geolocation'], rotation=90)

ax1.set_title('1.2.1. Proportion of population living below the national poverty line by Geolocation')
ax1.set_ylabel('Proportion');
ax1.set_xlabel('Geolocation');

#### Figure 1. Proportion of population living below the national poverty line by Geolocation (2015)
From the above figure, we can see that the bar of the **National Capital Region** is lower than the other regions. It has a proportion of lower than 10%, compared to the other graphs that look near 10% or higher. 

Using the same [`plot`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) function, let us also plot the data from 2018 into a bar graph.

In [ ]:
ax2 = data_2018['1.2.1. Proportion of population living below the national poverty line'].plot(figsize=(8, 6), kind='bar', width=0.5)

ax2.set_xticklabels(data_2018['Geolocation'], rotation=90)

ax2.set_title('1.2.1. Proportion of population living below the national poverty line by Geolocation')
ax2.set_ylabel('Proportion');
ax2.set_xlabel('Geolocation');

#### Figure 2. Proportion of population living below the national poverty line by Geolocation (2018)
For the `Year` 2018, we can see that the **National Capital Region** still has the shortest bar. Compared to the 2016 bar of  the region, this one is shorter.

From these two bar graphs, we can conclude that the **National Capital Region** has the lowest proportion value of the population living below the national poverty line for the years available in the dataset (i.e., 2016 and 2018).

### What education level (Junior or Senior High School) has a higher rate per region (2016 - 2018)?

### What year has the most adolescent birth rate?

## Conversion of DataFrame to File

## References
https://www.un.org/esa/sustdev/natlinfo/indicators/methodology_sheets/education/net_enrolment.pdf
https://psa.gov.ph/content/net-enrolment-ratio-ner-1